In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from sklearn.model_selection import train_test_split
from sklearn.base import TransformerMixin
pd.set_option('display.max_rows', 500)
pd.set_option('display.show_dimensions', True)
pd.set_option('display.max_columns', None)
%matplotlib inline
plt.style.use('fivethirtyeight')
warnings.simplefilter('ignore')

In [2]:
data_before_imputation = pd.read_csv('/home/yungshun/workspace/py/sklearn-aftygh-respiratory-therapy/datasets/data_before_imputation.csv')

In [3]:
data_before_imputation.head()

,病歷號,收件日期,收件時間,Temperature,pH,pCO2,pO2,HCO3,TCO2,ABE,SBE,SBC,SAT,Na,K,Mg,Cl,BUN (B),Creatinine(B),mode,peep,fio2,tidal volume,respiratory rate,minute ventilation,spo2
0,114400,20130125,714,37.0,7.267,59.0,459.4,27.1,28.9,-0.6,-0.1,24.0,100.0,141.4,4.71,NaN,NaN,48.1,2.05,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,114400,20130125,1027,37.0,7.281,59.3,94.0,28.1,30.0,0.5,1.1,24.8,96.0,141.4,4.71,NaN,NaN,48.1,2.05,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,114400,20130125,1250,37.0,7.287,54.0,136.5,26.0,27.6,-1.1,-0.9,23.5,98.7,141.4,4.71,NaN,NaN,48.1,2.05,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,114400,20130327,128,37.0,7.098,178.8,65.3,55.7,61.2,18.5,25.8,42.3,78.2,140.8,3.40,NaN,NaN,50.8,2.49,PCV,8.0,80.0,451.0,18.0,8.2,98.0
4,133750,20140317,914,37.0,7.373,36.7,358.5,20.9,22.0,-3.7,-4.4,21.5,99.8,161.0,4.00,NaN,NaN,73.0,5.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
data_before_imputation.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1132 entries, 0 to 1131
Data columns (total 26 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   病歷號                 1132 non-null   int64  
 1   收件日期                1132 non-null   int64  
 2   收件時間                1132 non-null   int64  
 3   Temperature         1132 non-null   float64
 4   pH                  1132 non-null   float64
 5   pCO2                1132 non-null   float64
 6   pO2                 1132 non-null   float64
 7   HCO3                1132 non-null   float64
 8   TCO2                1132 non-null   float64
 9   ABE                 1132 non-null   float64
 10  SBE                 1132 non-null   float64
 11  SBC                 1132 non-null   float64
 12  SAT                 1132 non-null   float64
 13  Na                  682 non-null    float64
 14  K                   733 non-null    float64
 15  Mg                  107 non-null    float64
 16  Cl    

In [5]:
data_before_imputation_describe = data_before_imputation.describe().T
data_before_imputation_describe

,count,mean,std,min,25%,50%,75%,max
病歷號,1132.0,7.518137e+07,3.427009e+07,1.144000e+05,9.203899e+07,92267205.00,9.244069e+07,9.930224e+07
收件日期,1132.0,2.012976e+07,6.121719e+03,2.012043e+07,2.012788e+07,20130623.00,2.013103e+07,2.014061e+07
收件時間,1132.0,1.137698e+03,5.969314e+02,1.000000e+00,6.465000e+02,1119.00,1.551000e+03,2.357000e+03
Temperature,1132.0,3.700000e+01,0.000000e+00,3.700000e+01,3.700000e+01,37.00,3.700000e+01,3.700000e+01
pH,1132.0,7.385461e+00,1.212951e-01,6.742000e+00,7.333000e+00,7.41,7.466250e+00,7.805000e+00
pCO2,1132.0,4.179629e+01,1.859072e+01,1.110000e+01,3.037500e+01,37.55,4.700000e+01,1.930000e+02
pO2,1132.0,1.286234e+02,9.372816e+01,1.740000e+01,6.890000e+01,96.05,1.554250e+02,5.787000e+02
HCO3,1132.0,2.397712e+01,6.703213e+00,1.500000e+00,1.960000e+01,23.60,2.790000e+01,5.630000e+01
TCO2,1132.0,2.525892e+01,7.080016e+00,1.800000e+00,2.060000e+01,24.90,2.930000e+01,6.120000e+01
ABE,1132.0,-8.045053e-01,6.274646e+00,-3.360000e+01,-4.400000e+00,-0.60,2.800000e+00,2.290000e+01


In [6]:
data_before_imputation_describe.to_csv('/home/yungshun/workspace/py/sklearn-aftygh-respiratory-therapy/datasets/data_before_imputation_describe.csv', encoding='utf-8', index=True)

In [7]:
data_before_imputation_corr = data_before_imputation.corr()
data_before_imputation_corr

,病歷號,收件日期,收件時間,Temperature,pH,pCO2,pO2,HCO3,TCO2,ABE,SBE,SBC,SAT,Na,K,Mg,Cl,BUN (B),Creatinine(B),peep,fio2,tidal volume,respiratory rate,minute ventilation,spo2
病歷號,1.000000,0.015971,0.021026,NaN,0.051739,-0.060851,-0.042759,-0.058086,-0.059809,-0.027287,-0.039669,-0.046652,0.002950,0.023689,-0.030757,-0.138654,0.237433,-0.127693,-0.071947,-0.156576,-0.134194,0.094591,0.130687,0.036139,0.066229
收件日期,0.015971,1.000000,0.005640,NaN,0.052147,-0.017205,0.009077,0.020454,0.017903,0.033567,0.036089,0.033877,0.009912,0.004003,0.008377,-0.209567,0.166756,0.027290,0.098744,0.097478,0.020142,-0.050967,-0.080342,0.032439,0.040394
收件時間,0.021026,0.005640,1.000000,NaN,-0.055889,-0.042788,0.110859,-0.106381,-0.104299,-0.111687,-0.114788,-0.109391,0.019520,0.012799,0.076342,0.111907,-0.008445,0.055627,0.003402,0.005545,0.127312,0.060786,-0.017615,0.104411,-0.034415
Temperature,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
pH,0.051739,0.052147,-0.055889,NaN,1.000000,-0.613459,0.038453,0.133354,0.076994,0.553216,0.394265,0.533207,0.308141,-0.024107,-0.333517,-0.246637,0.047769,-0.100313,-0.156605,-0.026492,-0.384225,-0.017260,-0.003096,-0.111207,0.223996
pCO2,-0.060851,-0.017205,-0.042788,NaN,-0.613459,1.000000,-0.102274,0.637627,0.684078,0.264726,0.425165,0.264881,-0.306889,-0.003376,0.085296,0.049720,-0.221836,-0.110107,-0.100218,0.188316,0.274776,-0.185331,-0.064077,-0.182406,-0.094274
pO2,-0.042759,0.009077,0.110859,NaN,0.038453,-0.102274,1.000000,-0.112988,-0.115172,-0.075757,-0.095264,-0.056139,0.454708,0.041993,-0.000843,-0.038567,-0.019876,0.116043,-0.007347,0.177298,0.385832,0.080926,-0.047626,0.112965,-0.158449
HCO3,-0.058086,0.020454,-0.106381,NaN,0.133354,0.637627,-0.112988,1.000000,0.998065,0.898355,0.963195,0.901052,-0.093984,-0.023069,-0.191739,-0.159833,-0.242607,-0.254357,-0.300022,0.254365,-0.032319,-0.329623,-0.091949,-0.427707,0.130395
TCO2,-0.059809,0.017903,-0.104299,NaN,0.076994,0.684078,-0.115172,0.998065,1.000000,0.871888,0.946165,0.874435,-0.113801,-0.022052,-0.174221,-0.146564,-0.246935,-0.249955,-0.291496,0.256711,-0.008503,-0.329401,-0.092606,-0.422255,0.115967
ABE,-0.027287,0.033567,-0.111687,NaN,0.553216,0.264726,-0.075757,0.898355,0.871888,1.000000,0.981920,0.993004,0.059041,-0.028759,-0.307282,-0.247640,-0.166973,-0.258249,-0.328586,0.193576,-0.206640,-0.272887,-0.078050,-0.402139,0.210327


In [8]:
data_before_imputation_corr.to_csv('/home/yungshun/workspace/py/sklearn-aftygh-respiratory-therapy/datasets/data_before_imputation_corr.csv', encoding='utf-8', index=True)

In [9]:
data_imputed = data_before_imputation.copy()
data_imputed['Na'] = data_imputed.groupby(['病歷號'])['Na'].transform(lambda x: x.fillna(method='bfill'))
data_imputed['K'] = data_imputed.groupby(['病歷號'])['K'].transform(lambda x: x.fillna(method='bfill'))
data_imputed['Mg'] = data_imputed.groupby(['病歷號'])['Mg'].transform(lambda x: x.fillna(method='bfill'))
data_imputed['Cl'] = data_imputed.groupby(['病歷號'])['Cl'].transform(lambda x: x.fillna(method='bfill'))
data_imputed['BUN (B)'] = data_imputed.groupby(['病歷號'])['BUN (B)'].transform(lambda x: x.fillna(method='bfill'))
data_imputed['Creatinine(B)'] = data_imputed.groupby(['病歷號'])['Creatinine(B)'].transform(lambda x: x.fillna(method='bfill'))
data_imputed['mode'] = data_imputed.groupby(['病歷號'])['mode'].transform(lambda x: x.fillna(method='bfill'))
data_imputed['peep'] = data_imputed.groupby(['病歷號'])['peep'].transform(lambda x: x.fillna(method='bfill'))
data_imputed['fio2'] = data_imputed.groupby(['病歷號'])['fio2'].transform(lambda x: x.fillna(method='bfill'))
data_imputed['tidal volume'] = data_imputed.groupby(['病歷號'])['tidal volume'].transform(lambda x: x.fillna(method='bfill'))
data_imputed['respiratory rate'] = data_imputed.groupby(['病歷號'])['respiratory rate'].transform(lambda x: x.fillna(method='bfill'))
data_imputed['minute ventilation'] = data_imputed.groupby(['病歷號'])['minute ventilation'].transform(lambda x: x.fillna(method='bfill'))
data_imputed['Na'] = data_imputed.groupby(['病歷號'])['Na'].transform(lambda x: x.fillna(method='ffill'))
data_imputed['K'] = data_imputed.groupby(['病歷號'])['K'].transform(lambda x: x.fillna(method='ffill'))
data_imputed['Mg'] = data_imputed.groupby(['病歷號'])['Mg'].transform(lambda x: x.fillna(method='ffill'))
data_imputed['Cl'] = data_imputed.groupby(['病歷號'])['Cl'].transform(lambda x: x.fillna(method='ffill'))
data_imputed['BUN (B)'] = data_imputed.groupby(['病歷號'])['BUN (B)'].transform(lambda x: x.fillna(method='ffill'))
data_imputed['Creatinine(B)'] = data_imputed.groupby(['病歷號'])['Creatinine(B)'].transform(lambda x: x.fillna(method='ffill'))
data_imputed['mode'] = data_imputed.groupby(['病歷號'])['mode'].transform(lambda x: x.fillna(method='ffill'))
data_imputed['peep'] = data_imputed.groupby(['病歷號'])['peep'].transform(lambda x: x.fillna(method='ffill'))
data_imputed['fio2'] = data_imputed.groupby(['病歷號'])['fio2'].transform(lambda x: x.fillna(method='ffill'))
data_imputed['tidal volume'] = data_imputed.groupby(['病歷號'])['tidal volume'].transform(lambda x: x.fillna(method='ffill'))
data_imputed['respiratory rate'] = data_imputed.groupby(['病歷號'])['respiratory rate'].transform(lambda x: x.fillna(method='ffill'))
data_imputed['minute ventilation'] = data_imputed.groupby(['病歷號'])['minute ventilation'].transform(lambda x: x.fillna(method='ffill'))
data_imputed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1132 entries, 0 to 1131
Data columns (total 26 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   病歷號                 1132 non-null   int64  
 1   收件日期                1132 non-null   int64  
 2   收件時間                1132 non-null   int64  
 3   Temperature         1132 non-null   float64
 4   pH                  1132 non-null   float64
 5   pCO2                1132 non-null   float64
 6   pO2                 1132 non-null   float64
 7   HCO3                1132 non-null   float64
 8   TCO2                1132 non-null   float64
 9   ABE                 1132 non-null   float64
 10  SBE                 1132 non-null   float64
 11  SBC                 1132 non-null   float64
 12  SAT                 1132 non-null   float64
 13  Na                  975 non-null    float64
 14  K                   990 non-null    float64
 15  Mg                  346 non-null    float64
 16  Cl    

In [10]:
data_imputed['Na'].fillna(data_imputed['Na'].mean(), inplace=True)
data_imputed['K'].fillna(data_imputed['K'].mean(), inplace=True)
data_imputed['Mg'].fillna(data_imputed['Mg'].mean(), inplace=True)
data_imputed['Cl'].fillna(data_imputed['Cl'].mean(), inplace=True)
data_imputed['BUN (B)'].fillna(data_imputed['BUN (B)'].mean(), inplace=True)
data_imputed['Creatinine(B)'].fillna(data_imputed['Creatinine(B)'].mean(), inplace=True)
data_imputed['mode'].fillna(data_imputed['mode'].mode()[0], inplace=True)
data_imputed['peep'].fillna(data_imputed['peep'].mean(), inplace=True)
data_imputed['fio2'].fillna(data_imputed['fio2'].mean(), inplace=True)
data_imputed['tidal volume'].fillna(data_imputed['tidal volume'].mean(), inplace=True)
data_imputed['respiratory rate'].fillna(data_imputed['respiratory rate'].mean(), inplace=True)
data_imputed['minute ventilation'].fillna(data_imputed['minute ventilation'].mean(), inplace=True)
data_imputed['spo2'].fillna(data_imputed['spo2'].mean(), inplace=True)
data_imputed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1132 entries, 0 to 1131
Data columns (total 26 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   病歷號                 1132 non-null   int64  
 1   收件日期                1132 non-null   int64  
 2   收件時間                1132 non-null   int64  
 3   Temperature         1132 non-null   float64
 4   pH                  1132 non-null   float64
 5   pCO2                1132 non-null   float64
 6   pO2                 1132 non-null   float64
 7   HCO3                1132 non-null   float64
 8   TCO2                1132 non-null   float64
 9   ABE                 1132 non-null   float64
 10  SBE                 1132 non-null   float64
 11  SBC                 1132 non-null   float64
 12  SAT                 1132 non-null   float64
 13  Na                  1132 non-null   float64
 14  K                   1132 non-null   float64
 15  Mg                  1132 non-null   float64
 16  Cl    

In [ ]:
data_imputed_corr = data_imputed.corr()
data_imputed_corr

In [ ]:
data_imputed_dropna = data_imputed.dropna()
data_imputed_dropna.info()

In [ ]:
data_dropped = data_before_imputation.drop(['Mg', 'Cl', 'mode'], axis=1)
data_dropped.info()

In [ ]:
data_dropped = data_dropped.dropna()
data_dropped.info()

In [ ]:
X = data_dropped[['Na', 'K', 'BUN (B)', 'Creatinine(B)', 'peep', 'fio2', 'tidal volume', 'respiratory rate', 'minute ventilation', 'spo2']].copy()
y = data_dropped['pH'].copy()

In [ ]:
X = data_imputed_dropna[['K', 'fio2','spo2']].copy()
y = data_imputed_dropna['pH'].copy()

In [ ]:
get_best_model_and_accuracy(lr, lr_params, X, y)

In [11]:
# X = data_imputed[['Na', 'K', 'Mg', 'Cl', 'BUN (B)', 'Creatinine(B)', 'mode', 'peep', 'fio2', 'tidal volume', 'respiratory rate', 'minute ventilation', 'spo2']].copy()
X = data_imputed[['Na', 'K', 'BUN (B)', 'Mg', 'Cl', 'Creatinine(B)', 'peep', 'fio2', 'tidal volume', 'respiratory rate', 'minute ventilation', 'spo2']].copy()
y = data_imputed['pH'].copy()

In [35]:
X = data_before_imputation[['Na', 'K', 'BUN (B)', 'Mg', 'Cl', 'Creatinine(B)', 'peep', 'fio2', 'tidal volume', 'respiratory rate', 'minute ventilation', 'spo2']].copy()
y = data_before_imputation['pH'].copy()

In [36]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1132 entries, 0 to 1131
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Na                  682 non-null    float64
 1   K                   733 non-null    float64
 2   BUN (B)             509 non-null    float64
 3   Mg                  107 non-null    float64
 4   Cl                  174 non-null    float64
 5   Creatinine(B)       583 non-null    float64
 6   peep                487 non-null    float64
 7   fio2                515 non-null    float64
 8   tidal volume        356 non-null    float64
 9   respiratory rate    486 non-null    float64
 10  minute ventilation  331 non-null    float64
 11  spo2                531 non-null    float64
dtypes: float64(12)
memory usage: 106.2 KB


In [37]:
y.info()

<class 'pandas.core.series.Series'>
RangeIndex: 1132 entries, 0 to 1131
Series name: pH
Non-Null Count  Dtype  
--------------  -----  
1132 non-null   float64
dtypes: float64(1)
memory usage: 9.0 KB


In [ ]:
class CustomDummifier(TransformerMixin):
    def __init__(self, cols=None):
        self.cols = cols
        
    def transform(self, X):
        return pd.get_dummies(X, columns=self.cols)
    
    def fit(self, *_):
        return self

In [ ]:
cd = CustomDummifier(cols=['mode'])

cd.fit_transform(X)

In [ ]:
def get_best_model_and_accuracy(model, params, X, y):
    # model - the model to grid search
    # params - the parameter set to try
    # error_score - if a parameter set raises an error, continue and set the performance as a big, fat 0
    grid = GridSearchCV(model, params, error_score=0.)
    # Fit the model and parameters
    grid.fit(X, y)           
    # Our classical metric for performance
    print("Best Accuracy: {}".format(grid.best_score_))
    # The best parameters that caused the best accuracy
    print("Best Parameters: {}".format(grid.best_params_))
    # The average time it took a model to fit to the data (in seconds)
    print("Average Time to Fit (s): {}".format(round(grid.cv_results_['mean_fit_time'].mean(), 3)))
    # The average time it took a model to predict out of sample data (in seconds)
    # This metric gives us insight into how this model will perform in real-time analysis
    print("Average Time to Score (s): {}".format(round(grid.cv_results_['mean_score_time'].mean(), 3)))

In [ ]:
# Import four machine learning models
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import HistGradientBoostingRegressor

In [ ]:
# Set up some parameters for our grid search
# We will start with four different machine learning models
# logistic regression, KNN, Decision Tree, and Random Forest
lr_params = {'C':[1e-1, 1e0, 1e1, 1e2], 'penalty':['none', 'l2']}
knn_params = {'n_neighbors': [1, 3, 5, 7]}
tree_params = {'max_depth': [None, 1, 3, 5, 7, 9, 11, 13, 15, 17, 19, 21]}
forest_params = {'n_estimators': [10, 50, 100], 'max_depth': [None, 1, 3, 5, 7]}
# hgb_params = {'max_iter': [100, 500, 1000, 1500], 'learning_rate': [0.1], 'max_depth': [25, 50, 75], 'l2_regularization': [0, 1.5]}

In [ ]:
# Instantiate the four machine learning models
linearRegression = LinearRegression()
lr = LogisticRegression()
knn = KNeighborsRegressor()
d_tree = DecisionTreeRegressor()
forest = RandomForestRegressor()
hgb = HistGradientBoostingRegressor()

In [ ]:
# Quick to fit, very slow to predict
get_best_model_and_accuracy(lr, lr_params, X, y)

In [ ]:
# Construct pipeline parameters based on the parameters for KNN on its own
knn_pipe_params = {'regressor__{}'.format(k): v for k, v in knn_params.items()}
print(knn_pipe_params)

# KNN requires a standard scalar due to using Euclidean distance as the main equation for predicting observations
knn_pipe = Pipeline([('scale', StandardScaler()), ('regressor', knn)])

# Quick to fit, very slow to predict
get_best_model_and_accuracy(knn_pipe, knn_pipe_params, X, y)

In [ ]:
get_best_model_and_accuracy(d_tree, tree_params, X, y)

In [ ]:
linearRegression = LinearRegression()
linearRegression.fit(X_train, y_train)
linearRegression.score(X_test, y_test)

In [ ]:
knn = KNeighborsRegressor()
knn.fit(X_train, y_train)
knn.score(X_test, y_test)

In [ ]:
d_tree = DecisionTreeRegressor()
d_tree.fit(X_train, y_train)
d_tree.score(X_test, y_test)

In [ ]:
hgb = HistGradientBoostingRegressor()
hgb.fit(X_train, y_train)
hgb.score(X_test, y_test)

In [38]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=20)
X.isnull().sum()

Na                     450
K                      399
BUN (B)                623
Mg                    1025
Cl                     958
Creatinine(B)          549
peep                   645
fio2                   617
tidal volume           776
respiratory rate       646
minute ventilation     801
spo2                   601
Length: 12, dtype: int64

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
#X_train['mode'] = le.fit_transform(X_train['mode'])
#X_test['mode'] = le.fit_transform(X_test['mode'])
y_train = le.fit_transform(y_train)
#y_test = le.fit_transform(y_test)

In [39]:
from sklearn import metrics
from sklearn.metrics import mean_squared_error
from sklearn.metrics import median_absolute_error
from sklearn.metrics import mean_squared_log_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import explained_variance_score
from sklearn.metrics import r2_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
import xgboost as xgb

# Parameter tunning
model = xgb.XGBRegressor()
param_dist = {"max_depth": [1, 2, 3, 4, 5],
              "n_estimators": [10, 50, 100, 200, 300, 500],
              "learning_rate": [0.5, 0.1, 0.01, 0.015]}
grid_search = GridSearchCV(model, param_grid=param_dist, cv=3, verbose=10, n_jobs=-1)
grid_search.fit(X_train, y_train)

grid_search.best_estimator_

Fitting 3 folds for each of 120 candidates, totalling 360 fits

[CV 1/3; 13/120] START learning_rate=0.5, max_depth=3, n_estimators=10..........
[CV 1/3; 13/120] END learning_rate=0.5, max_depth=3, n_estimators=10;, score=0.221 total time=   0.0s
[CV 2/3; 13/120] START learning_rate=0.5, max_depth=3, n_estimators=10..........
[CV 2/3; 13/120] END learning_rate=0.5, max_depth=3, n_estimators=10;, score=0.271 total time=   0.0s
[CV 1/3; 15/120] START learning_rate=0.5, max_depth=3, n_estimators=100.........
[CV 1/3; 15/120] END learning_rate=0.5, max_depth=3, n_estimators=100;, score=0.198 total time=   0.3s
[CV 2/3; 15/120] START learning_rate=0.5, max_depth=3, n_estimators=100.........
[CV 2/3; 15/120] END learning_rate=0.5, max_depth=3, n_estimators=100;, score=0.098 total time=   0.3s
[CV 2/3; 20/120] START learning_rate=0.5, max_depth=4, n_estimators=50..........
[CV 2/3; 20/120] END learning_rate=0.5, max_depth=4, n_estimators=50;, score=0.132 total time=   0.2s
[CV 3/3; 20/120] ST

[CV 3/3; 26/120] START learning_rate=0.5, max_depth=5, n_estimators=50..........
[CV 1/3; 117/120] START learning_rate=0.015, max_depth=5, n_estimators=100......
[CV 1/3; 117/120] END learning_rate=0.015, max_depth=5, n_estimators=100;, score=-0.557 total time=   0.3s
[CV 2/3; 119/120] START learning_rate=0.015, max_depth=5, n_estimators=300......
[CV 2/3; 119/120] END learning_rate=0.015, max_depth=5, n_estimators=300;, score=0.308 total time=   0.7s
[CV 1/3; 3/120] START learning_rate=0.5, max_depth=1, n_estimators=100..........
[CV 1/3; 3/120] END learning_rate=0.5, max_depth=1, n_estimators=100;, score=0.255 total time=   0.2s
[CV 1/3; 7/120] START learning_rate=0.5, max_depth=2, n_estimators=10...........
[CV 1/3; 7/120] END learning_rate=0.5, max_depth=2, n_estimators=10;, score=0.257 total time=   0.0s
[CV 1/3; 8/120] START learning_rate=0.5, max_depth=2, n_estimators=50...........
[CV 1/3; 8/120] END learning_rate=0.5, max_depth=2, n_estimators=50;, score=0.269 total time=   0.

[CV 3/3; 35/120] END learning_rate=0.1, max_depth=1, n_estimators=300;, score=0.144 total time=   0.7s
[CV 2/3; 115/120] START learning_rate=0.015, max_depth=5, n_estimators=10.......
[CV 2/3; 115/120] END learning_rate=0.015, max_depth=5, n_estimators=10;, score=-8.519 total time=   0.1s
[CV 1/3; 116/120] START learning_rate=0.015, max_depth=5, n_estimators=50.......
[CV 1/3; 116/120] END learning_rate=0.015, max_depth=5, n_estimators=50;, score=-3.064 total time=   0.2s
[CV 1/3; 118/120] START learning_rate=0.015, max_depth=5, n_estimators=200......
[CV 1/3; 118/120] END learning_rate=0.015, max_depth=5, n_estimators=200;, score=0.213 total time=   0.6s
[CV 1/3; 1/120] START learning_rate=0.5, max_depth=1, n_estimators=10...........
[CV 1/3; 1/120] END learning_rate=0.5, max_depth=1, n_estimators=10;, score=0.188 total time=   0.0s
[CV 2/3; 5/120] START learning_rate=0.5, max_depth=1, n_estimators=300..........
[CV 2/3; 5/120] END learning_rate=0.5, max_depth=1, n_estimators=300;, sc

[CV 2/3; 35/120] END learning_rate=0.1, max_depth=1, n_estimators=300;, score=0.041 total time=   0.8s
[CV 1/3; 42/120] START learning_rate=0.1, max_depth=2, n_estimators=500.........
[CV 1/3; 42/120] END learning_rate=0.1, max_depth=2, n_estimators=500;, score=0.298 total time=   1.3s
[CV 3/3; 54/120] START learning_rate=0.1, max_depth=4, n_estimators=500.........
[CV 3/3; 54/120] END learning_rate=0.1, max_depth=4, n_estimators=500;, score=0.213 total time=   1.5s
[CV 1/3; 64/120] START learning_rate=0.01, max_depth=1, n_estimators=200........
[CV 1/3; 64/120] END learning_rate=0.01, max_depth=1, n_estimators=200;, score=0.127 total time=   0.5s
[CV 1/3; 67/120] START learning_rate=0.01, max_depth=2, n_estimators=10.........
[CV 1/3; 67/120] END learning_rate=0.01, max_depth=2, n_estimators=10;, score=-0.009 total time=   0.0s
[CV 3/3; 67/120] START learning_rate=0.01, max_depth=2, n_estimators=10.........
[CV 3/3; 67/120] END learning_rate=0.01, max_depth=2, n_estimators=10;, score=

[CV 2/3; 51/120] START learning_rate=0.1, max_depth=4, n_estimators=100.........
[CV 3/3; 7/120] START learning_rate=0.5, max_depth=2, n_estimators=10...........
[CV 3/3; 7/120] END learning_rate=0.5, max_depth=2, n_estimators=10;, score=0.269 total time=   0.0s
[CV 3/3; 8/120] START learning_rate=0.5, max_depth=2, n_estimators=50...........
[CV 3/3; 8/120] END learning_rate=0.5, max_depth=2, n_estimators=50;, score=0.278 total time=   0.1s
[CV 3/3; 9/120] START learning_rate=0.5, max_depth=2, n_estimators=100..........
[CV 3/3; 9/120] END learning_rate=0.5, max_depth=2, n_estimators=100;, score=0.299 total time=   0.3s
[CV 1/3; 10/120] START learning_rate=0.5, max_depth=2, n_estimators=200.........
[CV 1/3; 10/120] END learning_rate=0.5, max_depth=2, n_estimators=200;, score=0.237 total time=   0.5s
[CV 3/3; 17/120] START learning_rate=0.5, max_depth=3, n_estimators=300.........
[CV 3/3; 17/120] END learning_rate=0.5, max_depth=3, n_estimators=300;, score=0.194 total time=   0.9s
[CV 

[CV 1/3; 60/120] START learning_rate=0.1, max_depth=5, n_estimators=500.........
[CV 3/3; 14/120] START learning_rate=0.5, max_depth=3, n_estimators=50..........
[CV 3/3; 14/120] END learning_rate=0.5, max_depth=3, n_estimators=50;, score=0.266 total time=   0.1s
[CV 1/3; 17/120] START learning_rate=0.5, max_depth=3, n_estimators=300.........
[CV 1/3; 17/120] END learning_rate=0.5, max_depth=3, n_estimators=300;, score=0.124 total time=   0.8s
[CV 2/3; 17/120] START learning_rate=0.5, max_depth=3, n_estimators=300.........
[CV 2/3; 17/120] END learning_rate=0.5, max_depth=3, n_estimators=300;, score=0.002 total time=   0.8s
[CV 2/3; 30/120] START learning_rate=0.5, max_depth=5, n_estimators=500.........
[CV 2/3; 30/120] END learning_rate=0.5, max_depth=5, n_estimators=500;, score=0.090 total time=   1.5s
[CV 3/3; 30/120] START learning_rate=0.5, max_depth=5, n_estimators=500.........
[CV 3/3; 30/120] END learning_rate=0.5, max_depth=5, n_estimators=500;, score=0.193 total time=   1.7s


[CV 3/3; 63/120] START learning_rate=0.01, max_depth=1, n_estimators=100........
[CV 2/3; 25/120] START learning_rate=0.5, max_depth=5, n_estimators=10..........
[CV 2/3; 25/120] END learning_rate=0.5, max_depth=5, n_estimators=10;, score=0.287 total time=   0.0s
[CV 2/3; 26/120] START learning_rate=0.5, max_depth=5, n_estimators=50..........
[CV 2/3; 26/120] END learning_rate=0.5, max_depth=5, n_estimators=50;, score=0.157 total time=   0.2s
[CV 3/3; 26/120] START learning_rate=0.5, max_depth=5, n_estimators=50..........
[CV 3/3; 26/120] END learning_rate=0.5, max_depth=5, n_estimators=50;, score=0.247 total time=   0.2s
[CV 2/3; 28/120] START learning_rate=0.5, max_depth=5, n_estimators=200.........
[CV 2/3; 28/120] END learning_rate=0.5, max_depth=5, n_estimators=200;, score=0.091 total time=   0.6s
[CV 3/3; 28/120] START learning_rate=0.5, max_depth=5, n_estimators=200.........
[CV 3/3; 28/120] END learning_rate=0.5, max_depth=5, n_estimators=200;, score=0.194 total time=   0.6s
[C

[CV 1/3; 96/120] START learning_rate=0.015, max_depth=1, n_estimators=500.......
[CV 3/3; 24/120] START learning_rate=0.5, max_depth=4, n_estimators=500.........
[CV 3/3; 24/120] END learning_rate=0.5, max_depth=4, n_estimators=500;, score=0.131 total time=   1.8s
[CV 2/3; 46/120] START learning_rate=0.1, max_depth=3, n_estimators=200.........
[CV 2/3; 46/120] END learning_rate=0.1, max_depth=3, n_estimators=200;, score=0.272 total time=   0.5s
[CV 3/3; 46/120] START learning_rate=0.1, max_depth=3, n_estimators=200.........
[CV 3/3; 46/120] END learning_rate=0.1, max_depth=3, n_estimators=200;, score=0.269 total time=   0.6s
[CV 2/3; 53/120] START learning_rate=0.1, max_depth=4, n_estimators=300.........
[CV 2/3; 53/120] END learning_rate=0.1, max_depth=4, n_estimators=300;, score=0.197 total time=   0.9s
[CV 2/3; 58/120] START learning_rate=0.1, max_depth=5, n_estimators=200.........
[CV 2/3; 58/120] END learning_rate=0.1, max_depth=5, n_estimators=200;, score=0.246 total time=   0.7s

[CV 2/3; 97/120] START learning_rate=0.015, max_depth=2, n_estimators=10........
[CV 3/3; 90/120] START learning_rate=0.01, max_depth=5, n_estimators=500........
[CV 3/3; 90/120] END learning_rate=0.01, max_depth=5, n_estimators=500;, score=0.328 total time=   1.5s
[CV 2/3; 102/120] START learning_rate=0.015, max_depth=2, n_estimators=500......
[CV 2/3; 102/120] END learning_rate=0.015, max_depth=2, n_estimators=500;, score=0.278 total time=   1.4s
[CV 3/3; 111/120] START learning_rate=0.015, max_depth=4, n_estimators=100......
[CV 3/3; 111/120] END learning_rate=0.015, max_depth=4, n_estimators=100;, score=-0.333 total time=   0.3s
[CV 3/3; 114/120] START learning_rate=0.015, max_depth=4, n_estimators=500......
[CV 3/3; 114/120] END learning_rate=0.015, max_depth=4, n_estimators=500;, score=0.319 total time=   1.4s
[CV 3/3; 2/120] START learning_rate=0.5, max_depth=1, n_estimators=50...........
[CV 3/3; 2/120] END learning_rate=0.5, max_depth=1, n_estimators=50;, score=0.256 total tim

XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
             importance_type=None, interaction_constraints='',
             learning_rate=0.015, max_bin=256, max_cat_to_onehot=4,
             max_delta_step=0, max_depth=2, max_leaves=0, min_child_weight=1,
             missing=nan, monotone_constraints='()', n_estimators=500, n_jobs=0,
             num_parallel_tree=1, predictor='auto', random_state=0, reg_alpha=0,
             reg_lambda=1, ...)

In [40]:
model = xgb.XGBRegressor(max_depth=2, 
                          min_child_weight=1, 
                          n_estimators=500, 
                          learning_rate=0.015)
model.fit(X_train, y_train)
predictions = model.predict(X_test)

In [43]:
#MSE = mean_squared_error(y_test, predictions)
#print("The score is %.5f" % MSE )
#mean_squared_error
MSE = mean_squared_error(y_test,predictions)
#print('MSE:',np.mean((y_test-predictions)**2))

RMSE = np.sqrt(mean_squared_error(y_test,predictions))

#median_absolute_error
Median_Absolute_Error = np.median(np.abs(y_test-predictions))
#print(median_absolute_error(y_test,predictions))

#mean_absolute_error
MAE = np.mean(np.abs(y_test-predictions))
#print(mean_absolute_error(y_test,predictions))

#mean_squared_log_error
MSLE = mean_squared_log_error(y_test,predictions)
#print(np.mean((np.log(y_test+1)-np.log(predictions+1))**2))

#explained_variance_score
Explained_Variance_Score = explained_variance_score(y_test,predictions)
#print(1-np.var(y_test-predictions)/np.var(y_test))

#r2_score
R2_Score = r2_score(y_test,predictions)
#print(1-(np.sum((y_test-predictions)**2))/np.sum((y_test -np.mean(y_test))**2))

xgb_model_scores_pH = pd.DataFrame(np.array([MSE, RMSE, Median_Absolute_Error, MAE, MSLE, Explained_Variance_Score, R2_Score]), columns=['pH scores'], index=['MSE', 'RMSE', 'Median Absolute Error', 'MAE', 'MSLE', 'Explained Variance Score', 'R2_Score'])
xgb_model_scores_pH

,pH scores
MSE,0.009908
RMSE,0.099537
Median Absolute Error,0.060708
MAE,0.072756
MSLE,0.000144
Explained Variance Score,0.285448
R2_Score,0.275400


In [44]:
xgb_model_scores_pH.to_csv('/home/yungshun/workspace/py/sklearn-aftygh-respiratory-therapy/datasets/xgb_scores_pH.csv', encoding='utf-8', index=True)

In [48]:
y_test

440     7.412
1075    7.428
1019    7.390
944     7.455
1105    7.383
747     7.430
404     7.476
999     7.323
811     7.358
292     7.515
120     7.492
884     7.416
641     7.428
159     7.494
627     7.322
913     7.337
213     7.461
238     7.384
703     7.335
71      7.409
977     7.444
439     7.424
1121    7.401
678     7.505
1004    7.415
239     7.277
1076    7.480
849     7.250
269     7.164
521     7.379
206     7.449
702     7.413
741     7.394
705     6.954
622     7.476
13      7.329
970     7.449
403     7.420
1006    7.448
214     7.512
18      7.476
128     7.506
942     7.468
175     7.440
1093    7.474
1023    7.358
1016    7.386
559     7.157
380     7.440
62      7.427
109     7.457
1122    7.396
650     7.387
1118    7.274
554     7.377
893     7.527
709     7.338
651     7.359
818     7.441
645     7.550
316     7.309
951     7.346
585     7.102
908     7.455
526     7.392
416     7.500
267     7.194
237     7.374
205     7.464
90      7.486
976     7.462
379   

In [49]:
y_test_label = y_test.apply(lambda x: 1 if x < 7.35 or x > 7.45 else 0)
y_test_label

440     0
1075    0
1019    0
944     1
1105    0
747     0
404     1
999     1
811     0
292     1
120     1
884     0
641     0
159     1
627     1
913     1
213     1
238     0
703     1
71      0
977     0
439     0
1121    0
678     1
1004    0
239     1
1076    1
849     1
269     1
521     0
206     0
702     0
741     0
705     1
622     1
13      1
970     0
403     0
1006    0
214     1
18      1
128     1
942     1
175     0
1093    1
1023    0
1016    0
559     1
380     0
62      0
109     1
1122    0
650     0
1118    1
554     0
893     1
709     1
651     0
818     0
645     1
316     1
951     1
585     1
908     1
526     0
416     1
267     1
237     0
205     1
90      1
976     1
379     0
334     0
340     1
947     1
504     1
784     1
370     1
58      0
506     0
955     0
432     1
751     1
810     0
142     1
677     1
696     1
956     1
1117    1
833     1
597     1
887     1
771     0
560     1
546     1
865     1
507     0
598     0
928     1
41      1


In [57]:
predictions

array([7.402555 , 7.388023 , 7.402891 , 7.314084 , 7.4100256, 7.3657603,
       7.412477 , 7.3722286, 7.398006 , 7.440707 , 7.414817 , 7.3819366,
       7.3922863, 7.440721 , 7.398242 , 7.333928 , 7.3762374, 7.339402 ,
       7.3440685, 7.4097257, 7.409269 , 7.435093 , 7.409269 , 7.421435 ,
       7.3982997, 7.376061 , 7.389476 , 7.3892827, 7.331592 , 7.405656 ,
       7.360911 , 7.366592 , 7.378311 , 7.3721185, 7.382636 , 7.3714843,
       7.390413 , 7.429943 , 7.420903 , 7.4137564, 7.3865438, 7.359861 ,
       7.409269 , 7.415337 , 7.4100256, 7.411702 , 7.411702 , 7.3129067,
       7.3684607, 7.3280597, 7.3932242, 7.3649282, 7.399153 , 7.375642 ,
       7.434697 , 7.2997675, 7.375239 , 7.3607693, 7.3998103, 7.4243617,
       7.3890214, 7.39969  , 7.026874 , 7.354524 , 7.411702 , 7.409269 ,
       7.372869 , 7.3897448, 7.3897448, 7.409269 , 7.4243617, 7.3684607,
       7.3536696, 7.4018636, 7.4109826, 7.409269 , 7.389476 , 7.4243617,
       7.4243617, 7.4053307, 7.4243617, 7.282853 , 

In [63]:
predictions_df = pd.DataFrame(predictions)
predictions_label = predictions_df[0].apply(lambda x: 1 if x < 7.35 or x > 7.45 else 0)
predictions_label.to_numpy()

array([0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0,
       1, 0, 1, 0])

In [64]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test_label, predictions_label)

0.5087719298245614

In [65]:
model.feature_importances_

array([0.06258159, 0.0956421 , 0.04874416, 0.04776149, 0.09409644,
       0.09121028, 0.07121475, 0.10901481, 0.08673513, 0.09242681,
       0.11697185, 0.0836006 ], dtype=float32)

In [66]:
xgb_feature_importances_pH = pd.DataFrame(np.array(model.feature_importances_), columns=['feature_importances'], index=['Na', 'K', 'BUN (B)', 'Mg', 'Cl', 'Creatinine(B)', 'peep', 'fio2', 'tidal volume', 'respiratory rate', 'minute ventilation', 'spo2'])
xgb_feature_importances_pH

,feature_importances
Na,0.062582
K,0.095642
BUN (B),0.048744
Mg,0.047761
Cl,0.094096
Creatinine(B),0.091210
peep,0.071215
fio2,0.109015
tidal volume,0.086735
respiratory rate,0.092427


In [67]:
xgb_feature_importances_pH.to_csv('/home/yungshun/workspace/py/sklearn-aftygh-respiratory-therapy/datasets/xgb_feature_importances_pH.csv', encoding='utf-8', index=True)

In [23]:
sum(xgb_feature_importances['feature_importances'])

1.0000000447034836

[CV 2/3; 2/120] START learning_rate=0.5, max_depth=1, n_estimators=50...........
[CV 2/3; 2/120] END learning_rate=0.5, max_depth=1, n_estimators=50;, score=0.075 total time=   0.1s
[CV 1/3; 6/120] START learning_rate=0.5, max_depth=1, n_estimators=500..........
[CV 1/3; 6/120] END learning_rate=0.5, max_depth=1, n_estimators=500;, score=0.093 total time=   1.2s
[CV 2/3; 16/120] START learning_rate=0.5, max_depth=3, n_estimators=200.........
[CV 2/3; 16/120] END learning_rate=0.5, max_depth=3, n_estimators=200;, score=-0.551 total time=   0.5s
[CV 2/3; 19/120] START learning_rate=0.5, max_depth=4, n_estimators=10..........
[CV 2/3; 19/120] END learning_rate=0.5, max_depth=4, n_estimators=10;, score=-0.117 total time=   0.0s
[CV 1/3; 20/120] START learning_rate=0.5, max_depth=4, n_estimators=50..........
[CV 1/3; 20/120] END learning_rate=0.5, max_depth=4, n_estimators=50;, score=0.030 total time=   0.2s
[CV 3/3; 21/120] START learning_rate=0.5, max_depth=4, n_estimators=100.........
[C

[CV 1/3; 3/120] START learning_rate=0.5, max_depth=1, n_estimators=100..........
[CV 1/3; 3/120] END learning_rate=0.5, max_depth=1, n_estimators=100;, score=0.111 total time=   0.3s
[CV 1/3; 8/120] START learning_rate=0.5, max_depth=2, n_estimators=50...........
[CV 1/3; 8/120] END learning_rate=0.5, max_depth=2, n_estimators=50;, score=0.089 total time=   0.1s
[CV 1/3; 9/120] START learning_rate=0.5, max_depth=2, n_estimators=100..........
[CV 1/3; 9/120] END learning_rate=0.5, max_depth=2, n_estimators=100;, score=0.052 total time=   0.3s
[CV 3/3; 11/120] START learning_rate=0.5, max_depth=2, n_estimators=300.........
[CV 3/3; 11/120] END learning_rate=0.5, max_depth=2, n_estimators=300;, score=-0.053 total time=   0.7s
[CV 3/3; 17/120] START learning_rate=0.5, max_depth=3, n_estimators=300.........
[CV 3/3; 17/120] END learning_rate=0.5, max_depth=3, n_estimators=300;, score=-0.192 total time=   0.8s
[CV 3/3; 23/120] START learning_rate=0.5, max_depth=4, n_estimators=300.........
[

[CV 2/3; 4/120] START learning_rate=0.5, max_depth=1, n_estimators=200..........
[CV 2/3; 4/120] END learning_rate=0.5, max_depth=1, n_estimators=200;, score=-0.094 total time=   0.5s
[CV 2/3; 9/120] START learning_rate=0.5, max_depth=2, n_estimators=100..........
[CV 2/3; 9/120] END learning_rate=0.5, max_depth=2, n_estimators=100;, score=-0.328 total time=   0.3s
[CV 1/3; 12/120] START learning_rate=0.5, max_depth=2, n_estimators=500.........
[CV 1/3; 12/120] END learning_rate=0.5, max_depth=2, n_estimators=500;, score=-0.104 total time=   1.2s
[CV 2/3; 21/120] START learning_rate=0.5, max_depth=4, n_estimators=100.........
[CV 2/3; 21/120] END learning_rate=0.5, max_depth=4, n_estimators=100;, score=-0.536 total time=   0.3s
[CV 2/3; 24/120] START learning_rate=0.5, max_depth=4, n_estimators=500.........
[CV 2/3; 24/120] END learning_rate=0.5, max_depth=4, n_estimators=500;, score=-0.591 total time=   1.4s
[CV 3/3; 34/120] START learning_rate=0.1, max_depth=1, n_estimators=200......

[CV 3/3; 4/120] START learning_rate=0.5, max_depth=1, n_estimators=200..........
[CV 3/3; 4/120] END learning_rate=0.5, max_depth=1, n_estimators=200;, score=0.014 total time=   0.5s
[CV 3/3; 9/120] START learning_rate=0.5, max_depth=2, n_estimators=100..........
[CV 3/3; 9/120] END learning_rate=0.5, max_depth=2, n_estimators=100;, score=0.008 total time=   0.3s
[CV 2/3; 12/120] START learning_rate=0.5, max_depth=2, n_estimators=500.........
[CV 2/3; 12/120] END learning_rate=0.5, max_depth=2, n_estimators=500;, score=-0.653 total time=   1.2s
[CV 1/3; 21/120] START learning_rate=0.5, max_depth=4, n_estimators=100.........
[CV 1/3; 21/120] END learning_rate=0.5, max_depth=4, n_estimators=100;, score=-0.039 total time=   0.3s
[CV 1/3; 24/120] START learning_rate=0.5, max_depth=4, n_estimators=500.........
[CV 1/3; 24/120] END learning_rate=0.5, max_depth=4, n_estimators=500;, score=-0.104 total time=   1.3s
[CV 1/3; 33/120] START learning_rate=0.1, max_depth=1, n_estimators=100........

In [70]:
# pCO2
X = data_imputed[['Na', 'K', 'BUN (B)', 'Mg', 'Cl', 'Creatinine(B)', 'peep', 'fio2', 'tidal volume', 'respiratory rate', 'minute ventilation', 'spo2']].copy()
y = data_imputed['pCO2'].copy()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=20)
X.isnull().sum()

# Parameter tunning
model = xgb.XGBRegressor()
param_dist = {"max_depth": [1, 2, 3, 4, 5],
              "n_estimators": [10, 50, 100, 200, 300, 500],
              "learning_rate": [0.5, 0.1, 0.01, 0.015]}
grid_search = GridSearchCV(model, param_grid=param_dist, cv=3, verbose=10, n_jobs=-1)
grid_search.fit(X_train, y_train)

grid_search.best_estimator_

Fitting 3 folds for each of 120 candidates, totalling 360 fits
[CV 1/3; 4/48] START learning_rate=0.1, max_depth=4, n_estimators=500...........
[CV 1/3; 4/48] END learning_rate=0.1, max_depth=4, n_estimators=500;, score=0.017 total time=   1.5s
[CV 1/3; 10/48] START learning_rate=0.1, max_depth=10, n_estimators=200.........
[CV 1/3; 10/48] END learning_rate=0.1, max_depth=10, n_estimators=200;, score=-0.022 total time=   0.8s
[CV 1/3; 13/48] START learning_rate=0.1, max_depth=20, n_estimators=100.........
[CV 1/3; 13/48] END learning_rate=0.1, max_depth=20, n_estimators=100;, score=-0.087 total time=   0.6s
[CV 3/3; 15/48] START learning_rate=0.1, max_depth=20, n_estimators=300.........
[CV 3/3; 15/48] END learning_rate=0.1, max_depth=20, n_estimators=300;, score=-0.004 total time=   1.2s
[CV 2/3; 21/48] START learning_rate=0.01, max_depth=5, n_estimators=100.........
[CV 2/3; 21/48] END learning_rate=0.01, max_depth=5, n_estimators=100;, score=-521.299 total time=   0.2s
[CV 2/3; 22/4

[CV 3/3; 84/120] START learning_rate=0.01, max_depth=4, n_estimators=500........[CV 3/3; 3/48] START learning_rate=0.1, max_depth=4, n_estimators=300...........
[CV 3/3; 3/48] END learning_rate=0.1, max_depth=4, n_estimators=300;, score=0.021 total time=   1.0s
[CV 3/3; 8/48] START learning_rate=0.1, max_depth=5, n_estimators=500...........
[CV 3/3; 8/48] END learning_rate=0.1, max_depth=5, n_estimators=500;, score=-0.028 total time=   1.6s
[CV 3/3; 14/48] START learning_rate=0.1, max_depth=20, n_estimators=200.........
[CV 3/3; 14/48] END learning_rate=0.1, max_depth=20, n_estimators=200;, score=-0.004 total time=   1.1s
[CV 2/3; 19/48] START learning_rate=0.01, max_depth=4, n_estimators=300.........
[CV 2/3; 19/48] END learning_rate=0.01, max_depth=4, n_estimators=300;, score=-9.892 total time=   0.7s
[CV 2/3; 23/48] START learning_rate=0.01, max_depth=5, n_estimators=300.........
[CV 2/3; 23/48] END learning_rate=0.01, max_depth=5, n_estimators=300;, score=-9.929 total time=   0.8s


[CV 3/3; 102/120] START learning_rate=0.015, max_depth=2, n_estimators=500......[CV 1/3; 1/48] START learning_rate=0.1, max_depth=4, n_estimators=100...........
[CV 1/3; 1/48] END learning_rate=0.1, max_depth=4, n_estimators=100;, score=0.155 total time=   0.3s
[CV 1/3; 5/48] START learning_rate=0.1, max_depth=5, n_estimators=100...........
[CV 1/3; 5/48] END learning_rate=0.1, max_depth=5, n_estimators=100;, score=0.084 total time=   0.4s
[CV 3/3; 6/48] START learning_rate=0.1, max_depth=5, n_estimators=200...........
[CV 3/3; 6/48] END learning_rate=0.1, max_depth=5, n_estimators=200;, score=0.042 total time=   0.6s
[CV 3/3; 9/48] START learning_rate=0.1, max_depth=10, n_estimators=100..........
[CV 3/3; 9/48] END learning_rate=0.1, max_depth=10, n_estimators=100;, score=0.050 total time=   0.4s
[CV 3/3; 12/48] START learning_rate=0.1, max_depth=10, n_estimators=500.........
[CV 3/3; 12/48] END learning_rate=0.1, max_depth=10, n_estimators=500;, score=0.009 total time=   1.8s
[CV 1/3

[CV 2/3; 111/120] START learning_rate=0.015, max_depth=4, n_estimators=100......[CV 3/3; 4/48] START learning_rate=0.1, max_depth=4, n_estimators=500...........
[CV 3/3; 4/48] END learning_rate=0.1, max_depth=4, n_estimators=500;, score=-0.020 total time=   1.6s
[CV 1/3; 11/48] START learning_rate=0.1, max_depth=10, n_estimators=300.........
[CV 1/3; 11/48] END learning_rate=0.1, max_depth=10, n_estimators=300;, score=-0.046 total time=   1.3s
[CV 1/3; 15/48] START learning_rate=0.1, max_depth=20, n_estimators=300.........
[CV 1/3; 15/48] END learning_rate=0.1, max_depth=20, n_estimators=300;, score=-0.098 total time=   1.2s
[CV 2/3; 20/48] START learning_rate=0.01, max_depth=4, n_estimators=500.........
[CV 2/3; 20/48] END learning_rate=0.01, max_depth=4, n_estimators=500;, score=-0.211 total time=   1.3s
[CV 2/3; 28/48] START learning_rate=0.01, max_depth=10, n_estimators=500........
[CV 2/3; 28/48] END learning_rate=0.01, max_depth=10, n_estimators=500;, score=-0.381 total time=   1

XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
             importance_type=None, interaction_constraints='',
             learning_rate=0.1, max_bin=256, max_cat_to_onehot=4,
             max_delta_step=0, max_depth=1, max_leaves=0, min_child_weight=1,
             missing=nan, monotone_constraints='()', n_estimators=50, n_jobs=0,
             num_parallel_tree=1, predictor='auto', random_state=0, reg_alpha=0,
             reg_lambda=1, ...)

In [71]:
model = xgb.XGBRegressor(max_depth=1, 
                          min_child_weight=1, 
                          n_estimators=50, 
                          learning_rate=0.1)
model.fit(X_train, y_train)
predictions = model.predict(X_test)

#mean_squared_error
MSE = mean_squared_error(y_test,predictions)
#print('MSE:',np.mean((y_test-predictions)**2))

RMSE = np.sqrt(mean_squared_error(y_test,predictions))

#median_absolute_error
Median_Absolute_Error = np.median(np.abs(y_test-predictions))
#print(median_absolute_error(y_test,predictions))

#mean_absolute_error
MAE = np.mean(np.abs(y_test-predictions))
#print(mean_absolute_error(y_test,predictions))

#mean_squared_log_error
MSLE = mean_squared_log_error(y_test,predictions)
#print(np.mean((np.log(y_test+1)-np.log(predictions+1))**2))

#explained_variance_score
Explained_Variance_Score = explained_variance_score(y_test,predictions)
#print(1-np.var(y_test-predictions)/np.var(y_test))

#r2_score
R2_Score = r2_score(y_test,predictions)
#print(1-(np.sum((y_test-predictions)**2))/np.sum((y_test -np.mean(y_test))**2))

xgb_model_scores_pCO2 = pd.DataFrame(np.array([MSE, RMSE, Median_Absolute_Error, MAE, MSLE, Explained_Variance_Score, R2_Score]), columns=['pCO2 scores'], index=['MSE', 'RMSE', 'Median Absolute Error', 'MAE', 'MSLE', 'Explained Variance Score', 'R2_Score'])
xgb_model_scores_pCO2

,pCO2 scores
MSE,275.379193
RMSE,16.594553
Median Absolute Error,7.807867
MAE,10.844881
MSLE,0.106856
Explained Variance Score,0.091882
R2_Score,0.089089


In [72]:
xgb_model_scores.to_csv('/home/yungshun/workspace/py/sklearn-aftygh-respiratory-therapy/datasets/xgb_scores_pCO2.csv', encoding='utf-8', index=True)

In [73]:
# pO2
X = data_imputed[['Na', 'K', 'BUN (B)', 'Mg', 'Cl', 'Creatinine(B)', 'peep', 'fio2', 'tidal volume', 'respiratory rate', 'minute ventilation', 'spo2']].copy()
y = data_imputed['pO2'].copy()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=20)
X.isnull().sum()

# Parameter tunning
model = xgb.XGBRegressor()
param_dist = {"max_depth": [1, 2, 3, 4, 5],
              "n_estimators": [10, 50, 100, 200, 300, 500],
              "learning_rate": [0.5, 0.1, 0.01, 0.015]}
grid_search = GridSearchCV(model, param_grid=param_dist, cv=3, verbose=10, n_jobs=-1)
grid_search.fit(X_train, y_train)

grid_search.best_estimator_

Fitting 3 folds for each of 120 candidates, totalling 360 fits
[CV 3/3; 2/48] START learning_rate=0.1, max_depth=4, n_estimators=200...........
[CV 3/3; 2/48] END learning_rate=0.1, max_depth=4, n_estimators=200;, score=0.047 total time=   0.7s
[CV 2/3; 7/48] START learning_rate=0.1, max_depth=5, n_estimators=300...........
[CV 2/3; 7/48] END learning_rate=0.1, max_depth=5, n_estimators=300;, score=-0.271 total time=   0.9s
[CV 2/3; 11/48] START learning_rate=0.1, max_depth=10, n_estimators=300.........
[CV 2/3; 11/48] END learning_rate=0.1, max_depth=10, n_estimators=300;, score=-0.423 total time=   1.3s
[CV 2/3; 15/48] START learning_rate=0.1, max_depth=20, n_estimators=300.........
[CV 2/3; 15/48] END learning_rate=0.1, max_depth=20, n_estimators=300;, score=-0.395 total time=   1.2s
[CV 1/3; 21/48] START learning_rate=0.01, max_depth=5, n_estimators=100.........
[CV 1/3; 21/48] END learning_rate=0.01, max_depth=5, n_estimators=100;, score=-422.156 total time=   0.2s
[CV 1/3; 22/48]

[CV 2/3; 4/120] START learning_rate=0.5, max_depth=1, n_estimators=200..........[CV 1/3; 2/48] START learning_rate=0.1, max_depth=4, n_estimators=200...........
[CV 1/3; 2/48] END learning_rate=0.1, max_depth=4, n_estimators=200;, score=0.109 total time=   0.6s
[CV 1/3; 7/48] START learning_rate=0.1, max_depth=5, n_estimators=300...........
[CV 1/3; 7/48] END learning_rate=0.1, max_depth=5, n_estimators=300;, score=-0.046 total time=   0.9s
[CV 3/3; 10/48] START learning_rate=0.1, max_depth=10, n_estimators=200.........
[CV 3/3; 10/48] END learning_rate=0.1, max_depth=10, n_estimators=200;, score=0.019 total time=   0.9s
[CV 3/3; 13/48] START learning_rate=0.1, max_depth=20, n_estimators=100.........
[CV 3/3; 13/48] END learning_rate=0.1, max_depth=20, n_estimators=100;, score=-0.000 total time=   0.6s
[CV 1/3; 16/48] START learning_rate=0.1, max_depth=20, n_estimators=500.........
[CV 1/3; 16/48] END learning_rate=0.1, max_depth=20, n_estimators=500;, score=-0.098 total time=   1.6s
[

[CV 2/3; 13/120] START learning_rate=0.5, max_depth=3, n_estimators=10..........[CV 1/3; 3/48] START learning_rate=0.1, max_depth=4, n_estimators=300...........
[CV 1/3; 3/48] END learning_rate=0.1, max_depth=4, n_estimators=300;, score=0.071 total time=   0.9s
[CV 1/3; 8/48] START learning_rate=0.1, max_depth=5, n_estimators=500...........
[CV 1/3; 8/48] END learning_rate=0.1, max_depth=5, n_estimators=500;, score=-0.102 total time=   1.7s
[CV 2/3; 14/48] START learning_rate=0.1, max_depth=20, n_estimators=200.........
[CV 2/3; 14/48] END learning_rate=0.1, max_depth=20, n_estimators=200;, score=-0.395 total time=   1.0s
[CV 2/3; 18/48] START learning_rate=0.01, max_depth=4, n_estimators=200.........
[CV 2/3; 18/48] END learning_rate=0.01, max_depth=4, n_estimators=200;, score=-70.840 total time=   0.5s
[CV 3/3; 20/48] START learning_rate=0.01, max_depth=4, n_estimators=500.........
[CV 3/3; 20/48] END learning_rate=0.01, max_depth=4, n_estimators=500;, score=0.017 total time=   1.3s


[CV 3/3; 22/120] START learning_rate=0.5, max_depth=4, n_estimators=200.........[CV 2/3; 1/48] START learning_rate=0.1, max_depth=4, n_estimators=100...........
[CV 2/3; 1/48] END learning_rate=0.1, max_depth=4, n_estimators=100;, score=-0.018 total time=   0.3s
[CV 2/3; 5/48] START learning_rate=0.1, max_depth=5, n_estimators=100...........
[CV 2/3; 5/48] END learning_rate=0.1, max_depth=5, n_estimators=100;, score=-0.100 total time=   0.4s
[CV 3/3; 7/48] START learning_rate=0.1, max_depth=5, n_estimators=300...........
[CV 3/3; 7/48] END learning_rate=0.1, max_depth=5, n_estimators=300;, score=0.007 total time=   0.9s
[CV 3/3; 11/48] START learning_rate=0.1, max_depth=10, n_estimators=300.........
[CV 3/3; 11/48] END learning_rate=0.1, max_depth=10, n_estimators=300;, score=0.010 total time=   1.5s
[CV 3/3; 16/48] START learning_rate=0.1, max_depth=20, n_estimators=500.........
[CV 3/3; 16/48] END learning_rate=0.1, max_depth=20, n_estimators=500;, score=-0.004 total time=   1.7s
[CV

[CV 3/3; 29/120] START learning_rate=0.5, max_depth=5, n_estimators=300.........[CV 2/3; 2/48] START learning_rate=0.1, max_depth=4, n_estimators=200...........
[CV 2/3; 2/48] END learning_rate=0.1, max_depth=4, n_estimators=200;, score=-0.146 total time=   0.6s
[CV 1/3; 6/48] START learning_rate=0.1, max_depth=5, n_estimators=200...........
[CV 1/3; 6/48] END learning_rate=0.1, max_depth=5, n_estimators=200;, score=0.004 total time=   0.6s
[CV 1/3; 9/48] START learning_rate=0.1, max_depth=10, n_estimators=100..........
[CV 1/3; 9/48] END learning_rate=0.1, max_depth=10, n_estimators=100;, score=0.044 total time=   0.4s
[CV 1/3; 12/48] START learning_rate=0.1, max_depth=10, n_estimators=500.........
[CV 1/3; 12/48] END learning_rate=0.1, max_depth=10, n_estimators=500;, score=-0.050 total time=   2.0s
[CV 1/3; 18/48] START learning_rate=0.01, max_depth=4, n_estimators=200.........
[CV 1/3; 18/48] END learning_rate=0.01, max_depth=4, n_estimators=200;, score=-56.477 total time=   0.5s
[

[CV 2/3; 41/120] START learning_rate=0.1, max_depth=2, n_estimators=300.........[CV 2/3; 4/48] START learning_rate=0.1, max_depth=4, n_estimators=500...........
[CV 2/3; 4/48] END learning_rate=0.1, max_depth=4, n_estimators=500;, score=-0.311 total time=   1.5s
[CV 2/3; 10/48] START learning_rate=0.1, max_depth=10, n_estimators=200.........
[CV 2/3; 10/48] END learning_rate=0.1, max_depth=10, n_estimators=200;, score=-0.400 total time=   0.9s
[CV 2/3; 13/48] START learning_rate=0.1, max_depth=20, n_estimators=100.........
[CV 2/3; 13/48] END learning_rate=0.1, max_depth=20, n_estimators=100;, score=-0.382 total time=   0.6s
[CV 2/3; 16/48] START learning_rate=0.1, max_depth=20, n_estimators=500.........
[CV 2/3; 16/48] END learning_rate=0.1, max_depth=20, n_estimators=500;, score=-0.395 total time=   1.6s
[CV 2/3; 24/48] START learning_rate=0.01, max_depth=5, n_estimators=500.........
[CV 2/3; 24/48] END learning_rate=0.01, max_depth=5, n_estimators=500;, score=-0.252 total time=   1.

XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
             importance_type=None, interaction_constraints='',
             learning_rate=0.015, max_bin=256, max_cat_to_onehot=4,
             max_delta_step=0, max_depth=3, max_leaves=0, min_child_weight=1,
             missing=nan, monotone_constraints='()', n_estimators=200, n_jobs=0,
             num_parallel_tree=1, predictor='auto', random_state=0, reg_alpha=0,
             reg_lambda=1, ...)

In [76]:
model = xgb.XGBRegressor(max_depth=3, 
                          min_child_weight=1, 
                          n_estimators=200, 
                          learning_rate=0.015)
model.fit(X_train, y_train)
predictions = model.predict(X_test)

#mean_squared_error
MSE = mean_squared_error(y_test,predictions)
#print('MSE:',np.mean((y_test-predictions)**2))

RMSE = np.sqrt(mean_squared_error(y_test,predictions))

#median_absolute_error
Median_Absolute_Error = np.median(np.abs(y_test-predictions))
#print(median_absolute_error(y_test,predictions))

#mean_absolute_error
MAE = np.mean(np.abs(y_test-predictions))
#print(mean_absolute_error(y_test,predictions))

#mean_squared_log_error
MSLE = mean_squared_log_error(y_test,predictions)
#print(np.mean((np.log(y_test+1)-np.log(predictions+1))**2))

#explained_variance_score
Explained_Variance_Score = explained_variance_score(y_test,predictions)
#print(1-np.var(y_test-predictions)/np.var(y_test))

#r2_score
R2_Score = r2_score(y_test,predictions)
#print(1-(np.sum((y_test-predictions)**2))/np.sum((y_test -np.mean(y_test))**2))

xgb_model_scores_pO2 = pd.DataFrame(np.array([MSE, RMSE, Median_Absolute_Error, MAE, MSLE, Explained_Variance_Score, R2_Score]), columns=['pO2 scores'], index=['MSE', 'RMSE', 'Median Absolute Error', 'MAE', 'MSLE', 'Explained Variance Score', 'R2_Score'])
xgb_model_scores_pO2

,pO2 scores
MSE,5342.043501
RMSE,73.089284
Median Absolute Error,33.283309
MAE,51.925976
MSLE,0.325814
Explained Variance Score,0.247436
R2_Score,0.247344


In [77]:
xgb_model_scores_pO2.to_csv('/home/yungshun/workspace/py/sklearn-aftygh-respiratory-therapy/datasets/xgb_scores_pO2.csv', encoding='utf-8', index=True)

In [78]:
# HCO3
X = data_imputed[['Na', 'K', 'BUN (B)', 'Mg', 'Cl', 'Creatinine(B)', 'peep', 'fio2', 'tidal volume', 'respiratory rate', 'minute ventilation', 'spo2']].copy()
y = data_imputed['HCO3'].copy()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=20)
X.isnull().sum()

# Parameter tunning
model = xgb.XGBRegressor()
param_dist = {"max_depth": [1, 2, 3, 4, 5],
              "n_estimators": [10, 50, 100, 200, 300, 500],
              "learning_rate": [0.5, 0.1, 0.01, 0.015]}
grid_search = GridSearchCV(model, param_grid=param_dist, cv=3, verbose=10, n_jobs=-1)
grid_search.fit(X_train, y_train)

grid_search.best_estimator_

Fitting 3 folds for each of 120 candidates, totalling 360 fits

[CV 1/3; 101/120] END learning_rate=0.015, max_depth=2, n_estimators=300;, score=0.045 total time=   0.7s
[CV 1/3; 107/120] START learning_rate=0.015, max_depth=3, n_estimators=300......
[CV 1/3; 107/120] END learning_rate=0.015, max_depth=3, n_estimators=300;, score=0.063 total time=   0.8s
[CV 1/3; 113/120] START learning_rate=0.015, max_depth=4, n_estimators=300......
[CV 1/3; 113/120] END learning_rate=0.015, max_depth=4, n_estimators=300;, score=0.066 total time=   1.0s
[CV 3/3; 119/120] START learning_rate=0.015, max_depth=5, n_estimators=300......
[CV 3/3; 119/120] END learning_rate=0.015, max_depth=5, n_estimators=300;, score=0.120 total time=   0.9s
[CV 2/3; 3/120] START learning_rate=0.5, max_depth=1, n_estimators=100..........
[CV 2/3; 3/120] END learning_rate=0.5, max_depth=1, n_estimators=100;, score=0.107 total time=   0.2s
[CV 1/3; 7/120] START learning_rate=0.5, max_depth=2, n_estimators=10...........
[CV 1

[CV 2/3; 2/120] END learning_rate=0.5, max_depth=1, n_estimators=50;, score=0.267 total time=   0.1s
[CV 3/3; 84/120] END learning_rate=0.01, max_depth=4, n_estimators=500;, score=0.106 total time=   1.5s
[CV 1/3; 93/120] START learning_rate=0.015, max_depth=1, n_estimators=100.......
[CV 1/3; 93/120] END learning_rate=0.015, max_depth=1, n_estimators=100;, score=-0.267 total time=   0.2s
[CV 1/3; 95/120] START learning_rate=0.015, max_depth=1, n_estimators=300.......
[CV 1/3; 95/120] END learning_rate=0.015, max_depth=1, n_estimators=300;, score=0.022 total time=   0.7s
[CV 2/3; 100/120] START learning_rate=0.015, max_depth=2, n_estimators=200......
[CV 2/3; 100/120] END learning_rate=0.015, max_depth=2, n_estimators=200;, score=-0.301 total time=   0.5s
[CV 3/3; 103/120] START learning_rate=0.015, max_depth=3, n_estimators=10.......
[CV 3/3; 103/120] END learning_rate=0.015, max_depth=3, n_estimators=10;, score=-3.573 total time=   0.0s
[CV 3/3; 105/120] START learning_rate=0.015, ma

[CV 3/3; 14/120] END learning_rate=0.5, max_depth=3, n_estimators=50;, score=0.293 total time=   0.2s
[CV 2/3; 111/120] END learning_rate=0.015, max_depth=4, n_estimators=100;, score=-0.300 total time=   0.6s
[CV 3/3; 114/120] START learning_rate=0.015, max_depth=4, n_estimators=500......
[CV 3/3; 114/120] END learning_rate=0.015, max_depth=4, n_estimators=500;, score=0.117 total time=   1.4s
[CV 1/3; 3/120] START learning_rate=0.5, max_depth=1, n_estimators=100..........
[CV 1/3; 3/120] END learning_rate=0.5, max_depth=1, n_estimators=100;, score=0.188 total time=   0.2s
[CV 3/3; 7/120] START learning_rate=0.5, max_depth=2, n_estimators=10...........
[CV 3/3; 7/120] END learning_rate=0.5, max_depth=2, n_estimators=10;, score=0.103 total time=   0.0s
[CV 3/3; 8/120] START learning_rate=0.5, max_depth=2, n_estimators=50...........
[CV 3/3; 8/120] END learning_rate=0.5, max_depth=2, n_estimators=50;, score=0.064 total time=   0.1s
[CV 3/3; 9/120] START learning_rate=0.5, max_depth=2, n_e

[CV 1/3; 43/120] END learning_rate=0.1, max_depth=3, n_estimators=10;, score=-1.615 total time=   0.0s
[CV 3/3; 102/120] END learning_rate=0.015, max_depth=2, n_estimators=500;, score=0.091 total time=   1.3s
[CV 3/3; 111/120] START learning_rate=0.015, max_depth=4, n_estimators=100......
[CV 3/3; 111/120] END learning_rate=0.015, max_depth=4, n_estimators=100;, score=-0.218 total time=   0.4s
[CV 2/3; 114/120] START learning_rate=0.015, max_depth=4, n_estimators=500......
[CV 2/3; 114/120] END learning_rate=0.015, max_depth=4, n_estimators=500;, score=-0.592 total time=   1.4s
[CV 1/3; 2/120] START learning_rate=0.5, max_depth=1, n_estimators=50...........
[CV 1/3; 2/120] END learning_rate=0.5, max_depth=1, n_estimators=50;, score=0.207 total time=   0.1s
[CV 2/3; 6/120] START learning_rate=0.5, max_depth=1, n_estimators=500..........
[CV 2/3; 6/120] END learning_rate=0.5, max_depth=1, n_estimators=500;, score=0.040 total time=   1.2s
[CV 2/3; 18/120] START learning_rate=0.5, max_dept

[CV 3/3; 69/120] END learning_rate=0.01, max_depth=2, n_estimators=100;, score=-1.374 total time=   0.3s
[CV 3/3; 52/120] END learning_rate=0.1, max_depth=4, n_estimators=200;, score=0.081 total time=   0.6s
[CV 3/3; 55/120] START learning_rate=0.1, max_depth=5, n_estimators=10..........
[CV 3/3; 55/120] END learning_rate=0.1, max_depth=5, n_estimators=10;, score=-0.133 total time=   0.0s
[CV 3/3; 56/120] START learning_rate=0.1, max_depth=5, n_estimators=50..........
[CV 3/3; 56/120] END learning_rate=0.1, max_depth=5, n_estimators=50;, score=0.081 total time=   0.2s
[CV 1/3; 58/120] START learning_rate=0.1, max_depth=5, n_estimators=200.........
[CV 1/3; 58/120] END learning_rate=0.1, max_depth=5, n_estimators=200;, score=0.147 total time=   0.6s
[CV 1/3; 61/120] START learning_rate=0.01, max_depth=1, n_estimators=10.........
[CV 1/3; 61/120] END learning_rate=0.01, max_depth=1, n_estimators=10;, score=-1.613 total time=   0.0s
[CV 2/3; 61/120] START learning_rate=0.01, max_depth=1, 

...

[CV 1/3; 83/120] END learning_rate=0.01, max_depth=4, n_estimators=300;, score=0.204 total time=   0.9s
[CV 3/3; 29/120] END learning_rate=0.5, max_depth=5, n_estimators=300;, score=-0.105 total time=   1.1s
[CV 1/3; 30/120] START learning_rate=0.5, max_depth=5, n_estimators=500.........
[CV 1/3; 30/120] END learning_rate=0.5, max_depth=5, n_estimators=500;, score=-0.100 total time=   1.6s
[CV 3/3; 46/120] START learning_rate=0.1, max_depth=3, n_estimators=200.........
[CV 3/3; 46/120] END learning_rate=0.1, max_depth=3, n_estimators=200;, score=0.076 total time=   0.5s
[CV 1/3; 49/120] START learning_rate=0.1, max_depth=4, n_estimators=10..........
[CV 1/3; 49/120] END learning_rate=0.1, max_depth=4, n_estimators=10;, score=0.034 total time=   0.0s
[CV 3/3; 49/120] START learning_rate=0.1, max_depth=4, n_estimators=10..........
[CV 3/3; 49/120] END learning_rate=0.1, max_depth=4, n_estimators=10;, score=-0.105 total time=   0.0s
[CV 3/3; 50/120] START learning_rate=0.1, max_depth=4, n

[CV 3/3; 89/120] END learning_rate=0.01, max_depth=5, n_estimators=300;, score=0.281 total time=   1.0s
[CV 3/3; 22/120] END learning_rate=0.5, max_depth=4, n_estimators=200;, score=-0.112 total time=   0.7s
[CV 2/3; 30/120] START learning_rate=0.5, max_depth=5, n_estimators=500.........
[CV 2/3; 30/120] END learning_rate=0.5, max_depth=5, n_estimators=500;, score=-0.147 total time=   1.9s
[CV 3/3; 30/120] START learning_rate=0.5, max_depth=5, n_estimators=500.........
[CV 3/3; 30/120] END learning_rate=0.5, max_depth=5, n_estimators=500;, score=-0.105 total time=   1.6s
[CV 3/3; 53/120] START learning_rate=0.1, max_depth=4, n_estimators=300.........
[CV 3/3; 53/120] END learning_rate=0.1, max_depth=4, n_estimators=300;, score=0.062 total time=   0.9s
[CV 3/3; 59/120] START learning_rate=0.1, max_depth=5, n_estimators=300.........
[CV 3/3; 59/120] END learning_rate=0.1, max_depth=5, n_estimators=300;, score=0.028 total time=   0.9s
[CV 2/3; 66/120] START learning_rate=0.01, max_depth=1

[CV 1/3; 109/120] END learning_rate=0.015, max_depth=4, n_estimators=10;, score=-10.386 total time=   0.1s
[CV 2/3; 13/120] END learning_rate=0.5, max_depth=3, n_estimators=10;, score=0.120 total time=   0.0s
[CV 1/3; 15/120] START learning_rate=0.5, max_depth=3, n_estimators=100.........
[CV 1/3; 15/120] END learning_rate=0.5, max_depth=3, n_estimators=100;, score=0.043 total time=   0.3s
[CV 2/3; 15/120] START learning_rate=0.5, max_depth=3, n_estimators=100.........
[CV 2/3; 15/120] END learning_rate=0.5, max_depth=3, n_estimators=100;, score=-0.029 total time=   0.3s
[CV 2/3; 20/120] START learning_rate=0.5, max_depth=4, n_estimators=50..........
[CV 2/3; 20/120] END learning_rate=0.5, max_depth=4, n_estimators=50;, score=-0.079 total time=   0.2s
[CV 3/3; 20/120] START learning_rate=0.5, max_depth=4, n_estimators=50..........
[CV 3/3; 20/120] END learning_rate=0.5, max_depth=4, n_estimators=50;, score=-0.021 total time=   0.2s
[CV 1/3; 23/120] START learning_rate=0.5, max_depth=4,

[CV 3/3; 117/120] END learning_rate=0.015, max_depth=5, n_estimators=100;, score=-0.317 total time=   0.3s
[CV 2/3; 41/120] END learning_rate=0.1, max_depth=2, n_estimators=300;, score=0.076 total time=   0.7s
[CV 3/3; 44/120] START learning_rate=0.1, max_depth=3, n_estimators=50..........
[CV 3/3; 44/120] END learning_rate=0.1, max_depth=3, n_estimators=50;, score=0.145 total time=   0.1s
[CV 3/3; 45/120] START learning_rate=0.1, max_depth=3, n_estimators=100.........
[CV 3/3; 45/120] END learning_rate=0.1, max_depth=3, n_estimators=100;, score=0.125 total time=   0.3s
[CV 3/3; 47/120] START learning_rate=0.1, max_depth=3, n_estimators=300.........
[CV 3/3; 47/120] END learning_rate=0.1, max_depth=3, n_estimators=300;, score=0.038 total time=   0.8s
[CV 2/3; 53/120] START learning_rate=0.1, max_depth=4, n_estimators=300.........
[CV 2/3; 53/120] END learning_rate=0.1, max_depth=4, n_estimators=300;, score=0.010 total time=   0.9s
[CV 2/3; 59/120] START learning_rate=0.1, max_depth=5, 

XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
             importance_type=None, interaction_constraints='',
             learning_rate=0.1, max_bin=256, max_cat_to_onehot=4,
             max_delta_step=0, max_depth=5, max_leaves=0, min_child_weight=1,
             missing=nan, monotone_constraints='()', n_estimators=100, n_jobs=0,
             num_parallel_tree=1, predictor='auto', random_state=0, reg_alpha=0,
             reg_lambda=1, ...)

In [79]:
model = xgb.XGBRegressor(max_depth=5, 
                          min_child_weight=1, 
                          n_estimators=100, 
                          learning_rate=0.1)
model.fit(X_train, y_train)
predictions = model.predict(X_test)

#mean_squared_error
MSE = mean_squared_error(y_test,predictions)
#print('MSE:',np.mean((y_test-predictions)**2))

RMSE = np.sqrt(mean_squared_error(y_test,predictions))

#median_absolute_error
Median_Absolute_Error = np.median(np.abs(y_test-predictions))
#print(median_absolute_error(y_test,predictions))

#mean_absolute_error
MAE = np.mean(np.abs(y_test-predictions))
#print(mean_absolute_error(y_test,predictions))

#mean_squared_log_error
MSLE = mean_squared_log_error(y_test,predictions)
#print(np.mean((np.log(y_test+1)-np.log(predictions+1))**2))

#explained_variance_score
Explained_Variance_Score = explained_variance_score(y_test,predictions)
#print(1-np.var(y_test-predictions)/np.var(y_test))

#r2_score
R2_Score = r2_score(y_test,predictions)
#print(1-(np.sum((y_test-predictions)**2))/np.sum((y_test -np.mean(y_test))**2))

xgb_model_scores_HCO3 = pd.DataFrame(np.array([MSE, RMSE, Median_Absolute_Error, MAE, MSLE, Explained_Variance_Score, R2_Score]), columns=['HCO3 scores'], index=['MSE', 'RMSE', 'Median Absolute Error', 'MAE', 'MSLE', 'Explained Variance Score', 'R2_Score'])
xgb_model_scores_HCO3

,HCO3 scores
MSE,20.838040
RMSE,4.564870
Median Absolute Error,2.695986
MAE,3.566885
MSLE,0.043968
Explained Variance Score,0.440520
R2_Score,0.436697


In [80]:
xgb_model_scores_HCO3.to_csv('/home/yungshun/workspace/py/sklearn-aftygh-respiratory-therapy/datasets/xgb_scores_HCO3.csv', encoding='utf-8', index=True)

In [81]:
# ABE
X = data_imputed[['Na', 'K', 'BUN (B)', 'Mg', 'Cl', 'Creatinine(B)', 'peep', 'fio2', 'tidal volume', 'respiratory rate', 'minute ventilation', 'spo2']].copy()
y = data_imputed['ABE'].copy()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=20)
X.isnull().sum()

# Parameter tunning
model = xgb.XGBRegressor()
param_dist = {"max_depth": [1, 2, 3, 4, 5],
              "n_estimators": [10, 50, 100, 200, 300, 500],
              "learning_rate": [0.5, 0.1, 0.01, 0.015]}
grid_search = GridSearchCV(model, param_grid=param_dist, cv=3, verbose=10, n_jobs=-1)
grid_search.fit(X_train, y_train)

grid_search.best_estimator_

Fitting 3 folds for each of 120 candidates, totalling 360 fits

[CV 3/3; 6/120] START learning_rate=0.5, max_depth=1, n_estimators=500..........
[CV 3/3; 6/120] END learning_rate=0.5, max_depth=1, n_estimators=500;, score=0.294 total time=   1.2s
[CV 3/3; 19/120] START learning_rate=0.5, max_depth=4, n_estimators=10..........
[CV 3/3; 19/120] END learning_rate=0.5, max_depth=4, n_estimators=10;, score=0.297 total time=   0.0s
[CV 1/3; 20/120] START learning_rate=0.5, max_depth=4, n_estimators=50..........
[CV 1/3; 20/120] END learning_rate=0.5, max_depth=4, n_estimators=50;, score=0.171 total time=   0.2s
[CV 1/3; 21/120] START learning_rate=0.5, max_depth=4, n_estimators=100.........
[CV 1/3; 21/120] END learning_rate=0.5, max_depth=4, n_estimators=100;, score=0.131 total time=   0.3s
[CV 2/3; 21/120] START learning_rate=0.5, max_depth=4, n_estimators=100.........
[CV 2/3; 21/120] END learning_rate=0.5, max_depth=4, n_estimators=100;, score=0.139 total time=   0.4s
[CV 3/3; 25/120] ST

[CV 1/3; 30/120] START learning_rate=0.5, max_depth=5, n_estimators=500.........
[CV 1/3; 17/120] START learning_rate=0.5, max_depth=3, n_estimators=300.........
[CV 1/3; 17/120] END learning_rate=0.5, max_depth=3, n_estimators=300;, score=0.168 total time=   0.9s
[CV 2/3; 17/120] START learning_rate=0.5, max_depth=3, n_estimators=300.........
[CV 2/3; 17/120] END learning_rate=0.5, max_depth=3, n_estimators=300;, score=0.042 total time=   1.0s
[CV 3/3; 29/120] START learning_rate=0.5, max_depth=5, n_estimators=300.........
[CV 3/3; 29/120] END learning_rate=0.5, max_depth=5, n_estimators=300;, score=0.248 total time=   0.9s
[CV 1/3; 30/120] START learning_rate=0.5, max_depth=5, n_estimators=500.........
[CV 1/3; 30/120] END learning_rate=0.5, max_depth=5, n_estimators=500;, score=0.164 total time=   1.5s
[CV 3/3; 46/120] START learning_rate=0.1, max_depth=3, n_estimators=200.........
[CV 3/3; 46/120] END learning_rate=0.1, max_depth=3, n_estimators=200;, score=0.323 total time=   0.5s

[CV 2/3; 79/120] START learning_rate=0.01, max_depth=4, n_estimators=10.........
[CV 1/3; 89/120] START learning_rate=0.01, max_depth=5, n_estimators=300........
[CV 1/3; 89/120] END learning_rate=0.01, max_depth=5, n_estimators=300;, score=0.210 total time=   1.0s
[CV 1/3; 96/120] START learning_rate=0.015, max_depth=1, n_estimators=500.......
[CV 1/3; 96/120] END learning_rate=0.015, max_depth=1, n_estimators=500;, score=0.180 total time=   1.1s
[CV 3/3; 103/120] START learning_rate=0.015, max_depth=3, n_estimators=10.......
[CV 3/3; 103/120] END learning_rate=0.015, max_depth=3, n_estimators=10;, score=-8.123 total time=   0.0s
[CV 3/3; 105/120] START learning_rate=0.015, max_depth=3, n_estimators=100......
[CV 3/3; 105/120] END learning_rate=0.015, max_depth=3, n_estimators=100;, score=-0.358 total time=   0.3s
[CV 1/3; 108/120] START learning_rate=0.015, max_depth=3, n_estimators=500......
[CV 1/3; 108/120] END learning_rate=0.015, max_depth=3, n_estimators=500;, score=0.258 total

[CV 1/3; 80/120] END learning_rate=0.01, max_depth=4, n_estimators=50;, score=0.122 total time=   0.2s
[CV 3/3; 43/120] START learning_rate=0.1, max_depth=3, n_estimators=10..........
[CV 3/3; 43/120] END learning_rate=0.1, max_depth=3, n_estimators=10;, score=-1.199 total time=   0.0s
[CV 2/3; 44/120] START learning_rate=0.1, max_depth=3, n_estimators=50..........
[CV 2/3; 44/120] END learning_rate=0.1, max_depth=3, n_estimators=50;, score=0.284 total time=   0.1s
[CV 3/3; 45/120] START learning_rate=0.1, max_depth=3, n_estimators=100.........
[CV 3/3; 45/120] END learning_rate=0.1, max_depth=3, n_estimators=100;, score=0.315 total time=   0.3s
[CV 1/3; 48/120] START learning_rate=0.1, max_depth=3, n_estimators=500.........
[CV 1/3; 48/120] END learning_rate=0.1, max_depth=3, n_estimators=500;, score=0.288 total time=   1.3s
[CV 1/3; 57/120] START learning_rate=0.1, max_depth=5, n_estimators=100.........
[CV 1/3; 57/120] END learning_rate=0.1, max_depth=5, n_estimators=100;, score=0.3

[CV 2/3; 96/120] START learning_rate=0.015, max_depth=1, n_estimators=500.......
[CV 1/3; 80/120] START learning_rate=0.01, max_depth=4, n_estimators=50.........
[CV 1/3; 80/120] END learning_rate=0.01, max_depth=4, n_estimators=50;, score=-5.126 total time=   0.2s
[CV 1/3; 82/120] START learning_rate=0.01, max_depth=4, n_estimators=200........
[CV 1/3; 82/120] END learning_rate=0.01, max_depth=4, n_estimators=200;, score=-0.056 total time=   0.6s
[CV 1/3; 85/120] START learning_rate=0.01, max_depth=5, n_estimators=10.........
[CV 1/3; 85/120] END learning_rate=0.01, max_depth=5, n_estimators=10;, score=-11.494 total time=   0.0s
[CV 1/3; 86/120] START learning_rate=0.01, max_depth=5, n_estimators=50.........
[CV 1/3; 86/120] END learning_rate=0.01, max_depth=5, n_estimators=50;, score=-5.131 total time=   0.2s
[CV 1/3; 88/120] START learning_rate=0.01, max_depth=5, n_estimators=200........
[CV 1/3; 88/120] END learning_rate=0.01, max_depth=5, n_estimators=200;, score=-0.061 total time

XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
             importance_type=None, interaction_constraints='',
             learning_rate=0.1, max_bin=256, max_cat_to_onehot=4,
             max_delta_step=0, max_depth=5, max_leaves=0, min_child_weight=1,
             missing=nan, monotone_constraints='()', n_estimators=50, n_jobs=0,
             num_parallel_tree=1, predictor='auto', random_state=0, reg_alpha=0,
             reg_lambda=1, ...)

In [82]:
model = xgb.XGBRegressor(max_depth=5, 
                          min_child_weight=1, 
                          n_estimators=50, 
                          learning_rate=0.1)
model.fit(X_train, y_train)
predictions = model.predict(X_test)

#mean_squared_error
MSE = mean_squared_error(y_test,predictions)
#print('MSE:',np.mean((y_test-predictions)**2))

RMSE = np.sqrt(mean_squared_error(y_test,predictions))

#median_absolute_error
Median_Absolute_Error = np.median(np.abs(y_test-predictions))
#print(median_absolute_error(y_test,predictions))

#mean_absolute_error
MAE = np.mean(np.abs(y_test-predictions))
#print(mean_absolute_error(y_test,predictions))

#mean_squared_log_error
#MSLE = mean_squared_log_error(y_test,predictions)
#print(np.mean((np.log(y_test+1)-np.log(predictions+1))**2))

#explained_variance_score
Explained_Variance_Score = explained_variance_score(y_test,predictions)
#print(1-np.var(y_test-predictions)/np.var(y_test))

#r2_score
R2_Score = r2_score(y_test,predictions)
#print(1-(np.sum((y_test-predictions)**2))/np.sum((y_test -np.mean(y_test))**2))

xgb_model_scores_ABE = pd.DataFrame(np.array([MSE, RMSE, Median_Absolute_Error, MAE, MSLE, Explained_Variance_Score, R2_Score]), columns=['ABE scores'], index=['MSE', 'RMSE', 'Median Absolute Error', 'MAE', 'MSLE', 'Explained Variance Score', 'R2_Score'])
xgb_model_scores_ABE

,ABE scores
MSE,19.644006
RMSE,4.432156
Median Absolute Error,2.218417
MAE,3.192705
MSLE,0.043968
Explained Variance Score,0.429537
R2_Score,0.425157


In [83]:
xgb_model_scores_ABE.to_csv('/home/yungshun/workspace/py/sklearn-aftygh-respiratory-therapy/datasets/xgb_scores_ABE.csv', encoding='utf-8', index=True)

In [85]:
# TCO2
X = data_imputed[['Na', 'K', 'BUN (B)', 'Mg', 'Cl', 'Creatinine(B)', 'peep', 'fio2', 'tidal volume', 'respiratory rate', 'minute ventilation', 'spo2']].copy()
y = data_imputed['TCO2'].copy()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=20)
X.isnull().sum()

# Parameter tunning
model = xgb.XGBRegressor()
param_dist = {"max_depth": [1, 2, 3, 4, 5],
              "n_estimators": [10, 50, 100, 200, 300, 500],
              "learning_rate": [0.5, 0.1, 0.01, 0.015]}
grid_search = GridSearchCV(model, param_grid=param_dist, cv=3, verbose=10, n_jobs=-1)
grid_search.fit(X_train, y_train)

grid_search.best_estimator_

Fitting 3 folds for each of 120 candidates, totalling 360 fits

[CV 3/3; 109/120] START learning_rate=0.015, max_depth=4, n_estimators=10.......
[CV 3/3; 109/120] END learning_rate=0.015, max_depth=4, n_estimators=10;, score=-8.112 total time=   0.0s
[CV 3/3; 110/120] START learning_rate=0.015, max_depth=4, n_estimators=50.......
[CV 3/3; 110/120] END learning_rate=0.015, max_depth=4, n_estimators=50;, score=-2.308 total time=   0.2s
[CV 1/3; 112/120] START learning_rate=0.015, max_depth=4, n_estimators=200......
[CV 1/3; 112/120] END learning_rate=0.015, max_depth=4, n_estimators=200;, score=0.204 total time=   0.6s
[CV 1/3; 115/120] START learning_rate=0.015, max_depth=5, n_estimators=10.......
[CV 1/3; 115/120] END learning_rate=0.015, max_depth=5, n_estimators=10;, score=-10.384 total time=   0.0s
[CV 3/3; 115/120] START learning_rate=0.015, max_depth=5, n_estimators=10.......
[CV 3/3; 115/120] END learning_rate=0.015, max_depth=5, n_estimators=10;, score=-8.119 total time=   0.0s


[CV 3/3; 7/120] START learning_rate=0.5, max_depth=2, n_estimators=10...........
[CV 3/3; 72/120] START learning_rate=0.01, max_depth=2, n_estimators=500........
[CV 3/3; 72/120] END learning_rate=0.01, max_depth=2, n_estimators=500;, score=0.249 total time=   1.2s
[CV 3/3; 81/120] START learning_rate=0.01, max_depth=4, n_estimators=100........
[CV 3/3; 81/120] END learning_rate=0.01, max_depth=4, n_estimators=100;, score=-1.326 total time=   0.3s
[CV 3/3; 84/120] START learning_rate=0.01, max_depth=4, n_estimators=500........
[CV 3/3; 84/120] END learning_rate=0.01, max_depth=4, n_estimators=500;, score=0.308 total time=   1.5s
[CV 3/3; 93/120] START learning_rate=0.015, max_depth=1, n_estimators=100.......
[CV 3/3; 93/120] END learning_rate=0.015, max_depth=1, n_estimators=100;, score=-0.470 total time=   0.2s
[CV 3/3; 95/120] START learning_rate=0.015, max_depth=1, n_estimators=300.......
[CV 3/3; 95/120] END learning_rate=0.015, max_depth=1, n_estimators=300;, score=0.168 total tim

[CV 2/3; 26/120] START learning_rate=0.5, max_depth=5, n_estimators=50..........
[CV 2/3; 96/120] START learning_rate=0.015, max_depth=1, n_estimators=500.......
[CV 2/3; 96/120] END learning_rate=0.015, max_depth=1, n_estimators=500;, score=0.206 total time=   1.1s
[CV 1/3; 105/120] START learning_rate=0.015, max_depth=3, n_estimators=100......
[CV 1/3; 105/120] END learning_rate=0.015, max_depth=3, n_estimators=100;, score=-0.543 total time=   0.3s
[CV 3/3; 107/120] START learning_rate=0.015, max_depth=3, n_estimators=300......
[CV 3/3; 107/120] END learning_rate=0.015, max_depth=3, n_estimators=300;, score=0.277 total time=   0.8s
[CV 3/3; 113/120] START learning_rate=0.015, max_depth=4, n_estimators=300......
[CV 3/3; 113/120] END learning_rate=0.015, max_depth=4, n_estimators=300;, score=0.299 total time=   0.9s
[CV 2/3; 119/120] START learning_rate=0.015, max_depth=5, n_estimators=300......
[CV 2/3; 119/120] END learning_rate=0.015, max_depth=5, n_estimators=300;, score=0.308 tot

[CV 1/3; 34/120] START learning_rate=0.1, max_depth=1, n_estimators=200.........
[CV 2/3; 71/120] START learning_rate=0.01, max_depth=2, n_estimators=300........
[CV 2/3; 71/120] END learning_rate=0.01, max_depth=2, n_estimators=300;, score=0.228 total time=   0.7s
[CV 2/3; 77/120] START learning_rate=0.01, max_depth=3, n_estimators=300........
[CV 2/3; 77/120] END learning_rate=0.01, max_depth=3, n_estimators=300;, score=0.259 total time=   0.8s
[CV 2/3; 84/120] START learning_rate=0.01, max_depth=4, n_estimators=500........
[CV 2/3; 84/120] END learning_rate=0.01, max_depth=4, n_estimators=500;, score=0.300 total time=   1.5s
[CV 1/3; 94/120] START learning_rate=0.015, max_depth=1, n_estimators=200.......
[CV 1/3; 94/120] END learning_rate=0.015, max_depth=1, n_estimators=200;, score=0.086 total time=   0.5s
[CV 1/3; 97/120] START learning_rate=0.015, max_depth=2, n_estimators=10........
[CV 1/3; 97/120] END learning_rate=0.015, max_depth=2, n_estimators=10;, score=-10.394 total time

[CV 2/3; 40/120] START learning_rate=0.1, max_depth=2, n_estimators=200.........
[CV 2/3; 2/120] START learning_rate=0.5, max_depth=1, n_estimators=50...........
[CV 2/3; 2/120] END learning_rate=0.5, max_depth=1, n_estimators=50;, score=0.299 total time=   0.1s
[CV 3/3; 6/120] START learning_rate=0.5, max_depth=1, n_estimators=500..........
[CV 3/3; 6/120] END learning_rate=0.5, max_depth=1, n_estimators=500;, score=0.268 total time=   1.2s
[CV 1/3; 19/120] START learning_rate=0.5, max_depth=4, n_estimators=10..........
[CV 1/3; 19/120] END learning_rate=0.5, max_depth=4, n_estimators=10;, score=0.243 total time=   0.0s
[CV 2/3; 19/120] START learning_rate=0.5, max_depth=4, n_estimators=10..........
[CV 2/3; 19/120] END learning_rate=0.5, max_depth=4, n_estimators=10;, score=0.245 total time=   0.0s
[CV 1/3; 21/120] START learning_rate=0.5, max_depth=4, n_estimators=100.........
[CV 1/3; 21/120] END learning_rate=0.5, max_depth=4, n_estimators=100;, score=0.178 total time=   0.4s
[CV 

[CV 2/3; 59/120] START learning_rate=0.1, max_depth=5, n_estimators=300.........
[CV 2/3; 120/120] START learning_rate=0.015, max_depth=5, n_estimators=500......
[CV 2/3; 120/120] END learning_rate=0.015, max_depth=5, n_estimators=500;, score=0.300 total time=   1.2s
[CV 2/3; 4/120] START learning_rate=0.5, max_depth=1, n_estimators=200..........
[CV 2/3; 4/120] END learning_rate=0.5, max_depth=1, n_estimators=200;, score=0.287 total time=   0.5s
[CV 2/3; 12/120] START learning_rate=0.5, max_depth=2, n_estimators=500.........
[CV 2/3; 12/120] END learning_rate=0.5, max_depth=2, n_estimators=500;, score=0.095 total time=   1.5s
[CV 3/3; 12/120] START learning_rate=0.5, max_depth=2, n_estimators=500.........
[CV 3/3; 12/120] END learning_rate=0.5, max_depth=2, n_estimators=500;, score=0.194 total time=   1.2s
[CV 2/3; 32/120] START learning_rate=0.1, max_depth=1, n_estimators=50..........
[CV 2/3; 32/120] END learning_rate=0.1, max_depth=1, n_estimators=50;, score=0.208 total time=   0.1

[CV 3/3; 62/120] END learning_rate=0.01, max_depth=1, n_estimators=50;, score=-4.072 total time=   0.1s
[CV 1/3; 75/120] START learning_rate=0.01, max_depth=3, n_estimators=100........
[CV 1/3; 75/120] END learning_rate=0.01, max_depth=3, n_estimators=100;, score=-1.798 total time=   0.3s
[CV 3/3; 77/120] START learning_rate=0.01, max_depth=3, n_estimators=300........
[CV 3/3; 77/120] END learning_rate=0.01, max_depth=3, n_estimators=300;, score=0.224 total time=   0.8s
[CV 2/3; 83/120] START learning_rate=0.01, max_depth=4, n_estimators=300........
[CV 2/3; 83/120] END learning_rate=0.01, max_depth=4, n_estimators=300;, score=0.294 total time=   0.8s
[CV 2/3; 89/120] START learning_rate=0.01, max_depth=5, n_estimators=300........
[CV 2/3; 89/120] END learning_rate=0.01, max_depth=5, n_estimators=300;, score=0.306 total time=   1.0s
[CV 3/3; 96/120] START learning_rate=0.015, max_depth=1, n_estimators=500.......
[CV 3/3; 96/120] END learning_rate=0.015, max_depth=1, n_estimators=500;, 

[CV 1/3; 81/120] START learning_rate=0.01, max_depth=4, n_estimators=100........
[CV 2/3; 79/120] END learning_rate=0.01, max_depth=4, n_estimators=10;, score=0.004 total time=   0.0s
[CV 2/3; 80/120] START learning_rate=0.01, max_depth=4, n_estimators=50.........
[CV 2/3; 80/120] END learning_rate=0.01, max_depth=4, n_estimators=50;, score=0.121 total time=   0.2s
[CV 1/3; 82/120] START learning_rate=0.01, max_depth=4, n_estimators=200........
[CV 1/3; 82/120] END learning_rate=0.01, max_depth=4, n_estimators=200;, score=0.277 total time=   0.7s
[CV 2/3; 85/120] START learning_rate=0.01, max_depth=5, n_estimators=10.........
[CV 2/3; 85/120] END learning_rate=0.01, max_depth=5, n_estimators=10;, score=0.011 total time=   0.1s
[CV 2/3; 86/120] START learning_rate=0.01, max_depth=5, n_estimators=50.........
[CV 2/3; 86/120] END learning_rate=0.01, max_depth=5, n_estimators=50;, score=0.146 total time=   0.3s
[CV 1/3; 88/120] START learning_rate=0.01, max_depth=5, n_estimators=200.......

[CV 3/3; 81/120] END learning_rate=0.01, max_depth=4, n_estimators=100;, score=-1.345 total time=   0.3s
[CV 1/3; 30/120] END learning_rate=0.5, max_depth=5, n_estimators=500;, score=0.113 total time=   1.5s
[CV 1/3; 47/120] START learning_rate=0.1, max_depth=3, n_estimators=300.........
[CV 1/3; 47/120] END learning_rate=0.1, max_depth=3, n_estimators=300;, score=0.292 total time=   0.9s
[CV 2/3; 47/120] START learning_rate=0.1, max_depth=3, n_estimators=300.........
[CV 2/3; 47/120] END learning_rate=0.1, max_depth=3, n_estimators=300;, score=0.249 total time=   0.8s
[CV 3/3; 56/120] START learning_rate=0.1, max_depth=5, n_estimators=50..........
[CV 3/3; 56/120] END learning_rate=0.1, max_depth=5, n_estimators=50;, score=0.264 total time=   0.2s
[CV 3/3; 58/120] START learning_rate=0.1, max_depth=5, n_estimators=200.........
[CV 3/3; 58/120] END learning_rate=0.1, max_depth=5, n_estimators=200;, score=0.243 total time=   0.6s
[CV 3/3; 61/120] START learning_rate=0.01, max_depth=1, n

XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
             importance_type=None, interaction_constraints='',
             learning_rate=0.015, max_bin=256, max_cat_to_onehot=4,
             max_delta_step=0, max_depth=5, max_leaves=0, min_child_weight=1,
             missing=nan, monotone_constraints='()', n_estimators=500, n_jobs=0,
             num_parallel_tree=1, predictor='auto', random_state=0, reg_alpha=0,
             reg_lambda=1, ...)

In [91]:
model = xgb.XGBRegressor(max_depth=5, 
                          min_child_weight=1, 
                          n_estimators=500, 
                          learning_rate=0.015)
model.fit(X_train, y_train)
predictions = model.predict(X_test)

#mean_squared_error
MSE = mean_squared_error(y_test,predictions)
#print('MSE:',np.mean((y_test-predictions)**2))

RMSE = np.sqrt(mean_squared_error(y_test,predictions))

#median_absolute_error
Median_Absolute_Error = np.median(np.abs(y_test-predictions))
#print(median_absolute_error(y_test,predictions))

#mean_absolute_error
MAE = np.mean(np.abs(y_test-predictions))
#print(mean_absolute_error(y_test,predictions))

#mean_squared_log_error
#MSLE = mean_squared_log_error(y_test,predictions)
#print(np.mean((np.log(y_test+1)-np.log(predictions+1))**2))

#explained_variance_score
Explained_Variance_Score = explained_variance_score(y_test,predictions)
#print(1-np.var(y_test-predictions)/np.var(y_test))

#r2_score
R2_Score = r2_score(y_test,predictions)
#print(1-(np.sum((y_test-predictions)**2))/np.sum((y_test -np.mean(y_test))**2))

xgb_model_scores_TCO2 = pd.DataFrame(np.array([MSE, RMSE, Median_Absolute_Error, MAE, MSLE, Explained_Variance_Score, R2_Score]), columns=['TCO2 scores'], index=['MSE', 'RMSE', 'Median Absolute Error', 'MAE', 'MSLE', 'Explained Variance Score', 'R2_Score'])
xgb_model_scores_TCO2

,TCO2 scores
MSE,138.702779
RMSE,11.777214
Median Absolute Error,3.785515
MAE,6.791927
MSLE,0.043968
Explained Variance Score,-0.054866
R2_Score,-0.055000


In [92]:
xgb_model_scores_TCO2.to_csv('/home/yungshun/workspace/py/sklearn-aftygh-respiratory-therapy/datasets/xgb_scores_TCO2.csv', encoding='utf-8', index=True)

In [88]:
# SAT
X = data_imputed[['Na', 'K', 'BUN (B)', 'Mg', 'Cl', 'Creatinine(B)', 'peep', 'fio2', 'tidal volume', 'respiratory rate', 'minute ventilation', 'spo2']].copy()
y = data_imputed['SAT'].copy()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=20)
X.isnull().sum()

# Parameter tunning
model = xgb.XGBRegressor()
param_dist = {"max_depth": [1, 2, 3, 4, 5],
              "n_estimators": [10, 50, 100, 200, 300, 500],
              "learning_rate": [0.5, 0.1, 0.01, 0.015]}
grid_search = GridSearchCV(model, param_grid=param_dist, cv=3, verbose=10, n_jobs=-1)
grid_search.fit(X_train, y_train)

grid_search.best_estimator_

Fitting 3 folds for each of 120 candidates, totalling 360 fits

[CV 3/3; 81/120] START learning_rate=0.01, max_depth=4, n_estimators=100........
[CV 3/3; 81/120] END learning_rate=0.01, max_depth=4, n_estimators=100;, score=0.190 total time=   0.3s
[CV 3/3; 84/120] START learning_rate=0.01, max_depth=4, n_estimators=500........
[CV 3/3; 84/120] END learning_rate=0.01, max_depth=4, n_estimators=500;, score=0.262 total time=   1.7s
[CV 2/3; 94/120] START learning_rate=0.015, max_depth=1, n_estimators=200.......
[CV 2/3; 94/120] END learning_rate=0.015, max_depth=1, n_estimators=200;, score=0.162 total time=   0.6s
[CV 1/3; 98/120] START learning_rate=0.015, max_depth=2, n_estimators=50........
[CV 1/3; 98/120] END learning_rate=0.015, max_depth=2, n_estimators=50;, score=0.116 total time=   0.1s
[CV 1/3; 99/120] START learning_rate=0.015, max_depth=2, n_estimators=100.......
[CV 1/3; 99/120] END learning_rate=0.015, max_depth=2, n_estimators=100;, score=0.178 total time=   0.3s
[CV 3/3; 

[CV 1/3; 25/120] START learning_rate=0.5, max_depth=5, n_estimators=10..........
[CV 2/3; 96/120] END learning_rate=0.015, max_depth=1, n_estimators=500;, score=0.239 total time=   1.1s
[CV 2/3; 103/120] START learning_rate=0.015, max_depth=3, n_estimators=10.......
[CV 2/3; 103/120] END learning_rate=0.015, max_depth=3, n_estimators=10;, score=0.009 total time=   0.0s
[CV 1/3; 105/120] START learning_rate=0.015, max_depth=3, n_estimators=100......
[CV 1/3; 105/120] END learning_rate=0.015, max_depth=3, n_estimators=100;, score=0.221 total time=   0.3s
[CV 3/3; 107/120] START learning_rate=0.015, max_depth=3, n_estimators=300......
[CV 3/3; 107/120] END learning_rate=0.015, max_depth=3, n_estimators=300;, score=0.275 total time=   1.0s
[CV 1/3; 114/120] START learning_rate=0.015, max_depth=4, n_estimators=500......
[CV 1/3; 114/120] END learning_rate=0.015, max_depth=4, n_estimators=500;, score=0.320 total time=   1.4s
[CV 1/3; 2/120] START learning_rate=0.5, max_depth=1, n_estimators=

[CV 1/3; 46/120] END learning_rate=0.1, max_depth=3, n_estimators=200;, score=-0.315 total time=   0.5s
[CV 3/3; 104/120] END learning_rate=0.015, max_depth=3, n_estimators=50;, score=0.135 total time=   0.2s
[CV 3/3; 106/120] START learning_rate=0.015, max_depth=3, n_estimators=200......
[CV 3/3; 106/120] END learning_rate=0.015, max_depth=3, n_estimators=200;, score=0.265 total time=   0.6s
[CV 1/3; 109/120] START learning_rate=0.015, max_depth=4, n_estimators=10.......
[CV 1/3; 109/120] END learning_rate=0.015, max_depth=4, n_estimators=10;, score=0.020 total time=   0.1s
[CV 3/3; 109/120] START learning_rate=0.015, max_depth=4, n_estimators=10.......
[CV 3/3; 109/120] END learning_rate=0.015, max_depth=4, n_estimators=10;, score=0.029 total time=   0.1s
[CV 3/3; 110/120] START learning_rate=0.015, max_depth=4, n_estimators=50.......
[CV 3/3; 110/120] END learning_rate=0.015, max_depth=4, n_estimators=50;, score=0.160 total time=   0.2s
[CV 3/3; 112/120] START learning_rate=0.015, m

[CV 3/3; 54/120] END learning_rate=0.1, max_depth=4, n_estimators=500;, score=-0.138 total time=   1.7s
[CV 2/3; 26/120] END learning_rate=0.5, max_depth=5, n_estimators=50;, score=0.214 total time=   0.2s
[CV 3/3; 26/120] START learning_rate=0.5, max_depth=5, n_estimators=50..........
[CV 3/3; 26/120] END learning_rate=0.5, max_depth=5, n_estimators=50;, score=0.269 total time=   0.2s
[CV 2/3; 28/120] START learning_rate=0.5, max_depth=5, n_estimators=200.........
[CV 2/3; 28/120] END learning_rate=0.5, max_depth=5, n_estimators=200;, score=0.170 total time=   0.7s
[CV 3/3; 28/120] START learning_rate=0.5, max_depth=5, n_estimators=200.........
[CV 3/3; 28/120] END learning_rate=0.5, max_depth=5, n_estimators=200;, score=0.214 total time=   0.7s
[CV 2/3; 36/120] START learning_rate=0.1, max_depth=1, n_estimators=500.........
[CV 2/3; 36/120] END learning_rate=0.1, max_depth=1, n_estimators=500;, score=0.280 total time=   1.2s
[CV 3/3; 36/120] START learning_rate=0.1, max_depth=1, n_es

[CV 2/3; 82/120] END learning_rate=0.01, max_depth=4, n_estimators=200;, score=-1.465 total time=   0.6s
[CV 3/3; 7/120] END learning_rate=0.5, max_depth=2, n_estimators=10;, score=0.270 total time=   0.0s
[CV 3/3; 8/120] START learning_rate=0.5, max_depth=2, n_estimators=50...........
[CV 3/3; 8/120] END learning_rate=0.5, max_depth=2, n_estimators=50;, score=0.306 total time=   0.1s
[CV 2/3; 10/120] START learning_rate=0.5, max_depth=2, n_estimators=200.........
[CV 2/3; 10/120] END learning_rate=0.5, max_depth=2, n_estimators=200;, score=0.262 total time=   0.5s
[CV 3/3; 10/120] START learning_rate=0.5, max_depth=2, n_estimators=200.........
[CV 3/3; 10/120] END learning_rate=0.5, max_depth=2, n_estimators=200;, score=0.266 total time=   0.5s
[CV 3/3; 21/120] START learning_rate=0.5, max_depth=4, n_estimators=100.........
[CV 3/3; 21/120] END learning_rate=0.5, max_depth=4, n_estimators=100;, score=0.222 total time=   0.3s
[CV 1/3; 22/120] START learning_rate=0.5, max_depth=4, n_est

[CV 3/3; 90/120] END learning_rate=0.01, max_depth=5, n_estimators=500;, score=0.004 total time=   1.6s
[CV 1/3; 89/120] START learning_rate=0.01, max_depth=5, n_estimators=300........
[CV 1/3; 89/120] END learning_rate=0.01, max_depth=5, n_estimators=300;, score=0.198 total time=   0.9s
[CV 2/3; 95/120] START learning_rate=0.015, max_depth=1, n_estimators=300.......
[CV 2/3; 95/120] END learning_rate=0.015, max_depth=1, n_estimators=300;, score=0.162 total time=   0.7s
[CV 1/3; 101/120] START learning_rate=0.015, max_depth=2, n_estimators=300......
[CV 1/3; 101/120] END learning_rate=0.015, max_depth=2, n_estimators=300;, score=0.205 total time=   0.7s
[CV 2/3; 107/120] START learning_rate=0.015, max_depth=3, n_estimators=300......
[CV 2/3; 107/120] END learning_rate=0.015, max_depth=3, n_estimators=300;, score=0.283 total time=   0.8s
[CV 2/3; 113/120] START learning_rate=0.015, max_depth=4, n_estimators=300......
[CV 2/3; 113/120] END learning_rate=0.015, max_depth=4, n_estimators=3

[CV 3/3; 105/120] START learning_rate=0.015, max_depth=3, n_estimators=100......
[CV 1/3; 34/120] END learning_rate=0.1, max_depth=1, n_estimators=200;, score=0.201 total time=   0.5s
[CV 3/3; 39/120] START learning_rate=0.1, max_depth=2, n_estimators=100.........
[CV 3/3; 39/120] END learning_rate=0.1, max_depth=2, n_estimators=100;, score=0.273 total time=   0.3s
[CV 1/3; 40/120] START learning_rate=0.1, max_depth=2, n_estimators=200.........
[CV 1/3; 40/120] END learning_rate=0.1, max_depth=2, n_estimators=200;, score=0.244 total time=   0.6s
[CV 1/3; 45/120] START learning_rate=0.1, max_depth=3, n_estimators=100.........
[CV 1/3; 45/120] END learning_rate=0.1, max_depth=3, n_estimators=100;, score=0.261 total time=   0.3s
[CV 2/3; 45/120] START learning_rate=0.1, max_depth=3, n_estimators=100.........
[CV 2/3; 45/120] END learning_rate=0.1, max_depth=3, n_estimators=100;, score=0.270 total time=   0.5s
[CV 2/3; 50/120] START learning_rate=0.1, max_depth=4, n_estimators=50..........

[CV 3/3; 96/120] END learning_rate=0.015, max_depth=1, n_estimators=500;, score=0.039 total time=   1.2s
[CV 2/3; 40/120] END learning_rate=0.1, max_depth=2, n_estimators=200;, score=0.275 total time=   0.5s
[CV 3/3; 40/120] START learning_rate=0.1, max_depth=2, n_estimators=200.........
[CV 3/3; 40/120] END learning_rate=0.1, max_depth=2, n_estimators=200;, score=0.293 total time=   0.5s
[CV 1/3; 47/120] START learning_rate=0.1, max_depth=3, n_estimators=300.........
[CV 1/3; 47/120] END learning_rate=0.1, max_depth=3, n_estimators=300;, score=0.286 total time=   0.8s
[CV 2/3; 47/120] START learning_rate=0.1, max_depth=3, n_estimators=300.........
[CV 2/3; 47/120] END learning_rate=0.1, max_depth=3, n_estimators=300;, score=0.240 total time=   0.8s
[CV 3/3; 55/120] START learning_rate=0.1, max_depth=5, n_estimators=10..........
[CV 3/3; 55/120] END learning_rate=0.1, max_depth=5, n_estimators=10;, score=-1.191 total time=   0.0s
[CV 3/3; 56/120] START learning_rate=0.1, max_depth=5, n

XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
             importance_type=None, interaction_constraints='',
             learning_rate=0.015, max_bin=256, max_cat_to_onehot=4,
             max_delta_step=0, max_depth=1, max_leaves=0, min_child_weight=1,
             missing=nan, monotone_constraints='()', n_estimators=500, n_jobs=0,
             num_parallel_tree=1, predictor='auto', random_state=0, reg_alpha=0,
             reg_lambda=1, ...)

In [89]:
model = xgb.XGBRegressor(max_depth=1, 
                          min_child_weight=1, 
                          n_estimators=500, 
                          learning_rate=0.015)
model.fit(X_train, y_train)
predictions = model.predict(X_test)

#mean_squared_error
MSE = mean_squared_error(y_test,predictions)
#print('MSE:',np.mean((y_test-predictions)**2))

RMSE = np.sqrt(mean_squared_error(y_test,predictions))

#median_absolute_error
Median_Absolute_Error = np.median(np.abs(y_test-predictions))
#print(median_absolute_error(y_test,predictions))

#mean_absolute_error
MAE = np.mean(np.abs(y_test-predictions))
#print(mean_absolute_error(y_test,predictions))

#mean_squared_log_error
#MSLE = mean_squared_log_error(y_test,predictions)
#print(np.mean((np.log(y_test+1)-np.log(predictions+1))**2))

#explained_variance_score
Explained_Variance_Score = explained_variance_score(y_test,predictions)
#print(1-np.var(y_test-predictions)/np.var(y_test))

#r2_score
R2_Score = r2_score(y_test,predictions)
#print(1-(np.sum((y_test-predictions)**2))/np.sum((y_test -np.mean(y_test))**2))

xgb_model_scores_SAT = pd.DataFrame(np.array([MSE, RMSE, Median_Absolute_Error, MAE, MSLE, Explained_Variance_Score, R2_Score]), columns=['SAT scores'], index=['MSE', 'RMSE', 'Median Absolute Error', 'MAE', 'MSLE', 'Explained Variance Score', 'R2_Score'])
xgb_model_scores_SAT

,SAT scores
MSE,134.501932
RMSE,11.597497
Median Absolute Error,4.831510
MAE,7.113534
MSLE,0.043968
Explained Variance Score,-0.023001
R2_Score,-0.023048


In [90]:
xgb_model_scores_SAT.to_csv('/home/yungshun/workspace/py/sklearn-aftygh-respiratory-therapy/datasets/xgb_scores_SAT.csv', encoding='utf-8', index=True)

In [93]:
# SBC
X = data_imputed[['Na', 'K', 'BUN (B)', 'Mg', 'Cl', 'Creatinine(B)', 'peep', 'fio2', 'tidal volume', 'respiratory rate', 'minute ventilation', 'spo2']].copy()
y = data_imputed['SBC'].copy()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=20)
X.isnull().sum()

# Parameter tunning
model = xgb.XGBRegressor()
param_dist = {"max_depth": [1, 2, 3, 4, 5],
              "n_estimators": [10, 50, 100, 200, 300, 500],
              "learning_rate": [0.5, 0.1, 0.01, 0.015]}
grid_search = GridSearchCV(model, param_grid=param_dist, cv=3, verbose=10, n_jobs=-1)
grid_search.fit(X_train, y_train)

grid_search.best_estimator_

Fitting 3 folds for each of 120 candidates, totalling 360 fits

[CV 1/3; 81/120] END learning_rate=0.01, max_depth=4, n_estimators=100;, score=-1.798 total time=   0.3s
[CV 2/3; 84/120] START learning_rate=0.01, max_depth=4, n_estimators=500........
[CV 2/3; 84/120] END learning_rate=0.01, max_depth=4, n_estimators=500;, score=0.300 total time=   1.5s
[CV 1/3; 94/120] START learning_rate=0.015, max_depth=1, n_estimators=200.......
[CV 1/3; 94/120] END learning_rate=0.015, max_depth=1, n_estimators=200;, score=0.077 total time=   0.5s
[CV 1/3; 97/120] START learning_rate=0.015, max_depth=2, n_estimators=10........
[CV 1/3; 97/120] END learning_rate=0.015, max_depth=2, n_estimators=10;, score=-10.301 total time=   0.0s
[CV 1/3; 98/120] START learning_rate=0.015, max_depth=2, n_estimators=50........
[CV 1/3; 98/120] END learning_rate=0.015, max_depth=2, n_estimators=50;, score=-3.048 total time=   0.1s
[CV 1/3; 99/120] START learning_rate=0.015, max_depth=2, n_estimators=100.......
[CV 1/

[CV 2/3; 8/120] END learning_rate=0.5, max_depth=2, n_estimators=50;, score=0.259 total time=   0.1s
[CV 1/3; 25/120] END learning_rate=0.5, max_depth=5, n_estimators=10;, score=-0.366 total time=   0.0s
[CV 2/3; 25/120] START learning_rate=0.5, max_depth=5, n_estimators=10..........
[CV 2/3; 25/120] END learning_rate=0.5, max_depth=5, n_estimators=10;, score=-0.169 total time=   0.0s
[CV 2/3; 26/120] START learning_rate=0.5, max_depth=5, n_estimators=50..........
[CV 2/3; 26/120] END learning_rate=0.5, max_depth=5, n_estimators=50;, score=-0.333 total time=   0.2s
[CV 3/3; 26/120] START learning_rate=0.5, max_depth=5, n_estimators=50..........
[CV 3/3; 26/120] END learning_rate=0.5, max_depth=5, n_estimators=50;, score=-0.101 total time=   0.2s
[CV 2/3; 28/120] START learning_rate=0.5, max_depth=5, n_estimators=200.........
[CV 2/3; 28/120] END learning_rate=0.5, max_depth=5, n_estimators=200;, score=-0.416 total time=   0.6s
[CV 3/3; 28/120] START learning_rate=0.5, max_depth=5, n_es

[CV 3/3; 29/120] END learning_rate=0.5, max_depth=5, n_estimators=300;, score=0.151 total time=   1.2s
[CV 3/3; 84/120] START learning_rate=0.01, max_depth=4, n_estimators=500........
[CV 3/3; 84/120] END learning_rate=0.01, max_depth=4, n_estimators=500;, score=0.300 total time=   1.5s
[CV 3/3; 93/120] START learning_rate=0.015, max_depth=1, n_estimators=100.......
[CV 3/3; 93/120] END learning_rate=0.015, max_depth=1, n_estimators=100;, score=-0.478 total time=   0.2s
[CV 1/3; 96/120] START learning_rate=0.015, max_depth=1, n_estimators=500.......
[CV 1/3; 96/120] END learning_rate=0.015, max_depth=1, n_estimators=500;, score=0.170 total time=   1.1s
[CV 3/3; 103/120] START learning_rate=0.015, max_depth=3, n_estimators=10.......
[CV 3/3; 103/120] END learning_rate=0.015, max_depth=3, n_estimators=10;, score=-8.185 total time=   0.0s
[CV 2/3; 105/120] START learning_rate=0.015, max_depth=3, n_estimators=100......
[CV 2/3; 105/120] END learning_rate=0.015, max_depth=3, n_estimators=10

[CV 2/3; 54/120] START learning_rate=0.1, max_depth=4, n_estimators=500.........
[CV 3/3; 51/120] START learning_rate=0.1, max_depth=4, n_estimators=100.........
[CV 3/3; 51/120] END learning_rate=0.1, max_depth=4, n_estimators=100;, score=-0.025 total time=   0.3s
[CV 3/3; 53/120] START learning_rate=0.1, max_depth=4, n_estimators=300.........
[CV 3/3; 53/120] END learning_rate=0.1, max_depth=4, n_estimators=300;, score=-0.089 total time=   0.9s
[CV 1/3; 58/120] START learning_rate=0.1, max_depth=5, n_estimators=200.........
[CV 1/3; 58/120] END learning_rate=0.1, max_depth=5, n_estimators=200;, score=-0.492 total time=   0.6s
[CV 1/3; 61/120] START learning_rate=0.01, max_depth=1, n_estimators=10.........
[CV 1/3; 61/120] END learning_rate=0.01, max_depth=1, n_estimators=10;, score=-73.275 total time=   0.0s
[CV 3/3; 61/120] START learning_rate=0.01, max_depth=1, n_estimators=10.........
[CV 3/3; 61/120] END learning_rate=0.01, max_depth=1, n_estimators=10;, score=-49.431 total time=

XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
             importance_type=None, interaction_constraints='',
             learning_rate=0.015, max_bin=256, max_cat_to_onehot=4,
             max_delta_step=0, max_depth=4, max_leaves=0, min_child_weight=1,
             missing=nan, monotone_constraints='()', n_estimators=500, n_jobs=0,
             num_parallel_tree=1, predictor='auto', random_state=0, reg_alpha=0,
             reg_lambda=1, ...)

In [95]:
model = xgb.XGBRegressor(max_depth=4, 
                          min_child_weight=1, 
                          n_estimators=500, 
                          learning_rate=0.015)
model.fit(X_train, y_train)
predictions = model.predict(X_test)

#mean_squared_error
MSE = mean_squared_error(y_test,predictions)
#print('MSE:',np.mean((y_test-predictions)**2))

RMSE = np.sqrt(mean_squared_error(y_test,predictions))

#median_absolute_error
Median_Absolute_Error = np.median(np.abs(y_test-predictions))
#print(median_absolute_error(y_test,predictions))

#mean_absolute_error
MAE = np.mean(np.abs(y_test-predictions))
#print(mean_absolute_error(y_test,predictions))

#mean_squared_log_error
#MSLE = mean_squared_log_error(y_test,predictions)
#print(np.mean((np.log(y_test+1)-np.log(predictions+1))**2))

#explained_variance_score
Explained_Variance_Score = explained_variance_score(y_test,predictions)
#print(1-np.var(y_test-predictions)/np.var(y_test))

#r2_score
R2_Score = r2_score(y_test,predictions)
#print(1-(np.sum((y_test-predictions)**2))/np.sum((y_test -np.mean(y_test))**2))

xgb_model_scores_SBC = pd.DataFrame(np.array([MSE, RMSE, Median_Absolute_Error, MAE, MSLE, Explained_Variance_Score, R2_Score]), columns=['SBC scores'], index=['MSE', 'RMSE', 'Median Absolute Error', 'MAE', 'MSLE', 'Explained Variance Score', 'R2_Score'])
xgb_model_scores_SBC

,SBC scores
MSE,12.987442
RMSE,3.603809
Median Absolute Error,2.111449
MAE,2.667267
MSLE,0.043968
Explained Variance Score,0.453928
R2_Score,0.450130


In [96]:
xgb_model_scores_SBC.to_csv('/home/yungshun/workspace/py/sklearn-aftygh-respiratory-therapy/datasets/xgb_scores_SBC.csv', encoding='utf-8', index=True)


[CV 2/3; 54/120] END learning_rate=0.1, max_depth=4, n_estimators=500;, score=0.130 total time=   1.4s
[CV 2/3; 63/120] START learning_rate=0.01, max_depth=1, n_estimators=100........
[CV 2/3; 63/120] END learning_rate=0.01, max_depth=1, n_estimators=100;, score=-2.753 total time=   0.3s
[CV 2/3; 65/120] START learning_rate=0.01, max_depth=1, n_estimators=300........
[CV 2/3; 65/120] END learning_rate=0.01, max_depth=1, n_estimators=300;, score=0.112 total time=   0.7s
[CV 3/3; 69/120] START learning_rate=0.01, max_depth=2, n_estimators=100........
[CV 3/3; 69/120] END learning_rate=0.01, max_depth=2, n_estimators=100;, score=-2.144 total time=   0.3s
[CV 3/3; 72/120] START learning_rate=0.01, max_depth=2, n_estimators=500........
[CV 3/3; 72/120] END learning_rate=0.01, max_depth=2, n_estimators=500;, score=0.241 total time=   1.3s
[CV 3/3; 79/120] START learning_rate=0.01, max_depth=4, n_estimators=10.........
[CV 3/3; 79/120] END learning_rate=0.01, max_depth=4, n_estimators=10;, s


[CV 1/3; 66/120] START learning_rate=0.01, max_depth=1, n_estimators=500........
[CV 1/3; 66/120] END learning_rate=0.01, max_depth=1, n_estimators=500;, score=0.001 total time=   1.3s
[CV 2/3; 74/120] START learning_rate=0.01, max_depth=3, n_estimators=50.........
[CV 2/3; 74/120] END learning_rate=0.01, max_depth=3, n_estimators=50;, score=-27.140 total time=   0.1s
[CV 2/3; 76/120] START learning_rate=0.01, max_depth=3, n_estimators=200........
[CV 2/3; 76/120] END learning_rate=0.01, max_depth=3, n_estimators=200;, score=-1.426 total time=   0.6s
[CV 1/3; 79/120] START learning_rate=0.01, max_depth=4, n_estimators=10.........
[CV 1/3; 79/120] END learning_rate=0.01, max_depth=4, n_estimators=10;, score=-73.265 total time=   0.1s
[CV 1/3; 80/120] START learning_rate=0.01, max_depth=4, n_estimators=50.........
[CV 1/3; 80/120] END learning_rate=0.01, max_depth=4, n_estimators=50;, score=-32.928 total time=   0.2s
[CV 3/3; 81/120] START learning_rate=0.01, max_depth=4, n_estimators=1


[CV 3/3; 2/120] START learning_rate=0.5, max_depth=1, n_estimators=50...........
[CV 3/3; 2/120] END learning_rate=0.5, max_depth=1, n_estimators=50;, score=0.244 total time=   0.1s
[CV 3/3; 6/120] START learning_rate=0.5, max_depth=1, n_estimators=500..........
[CV 3/3; 6/120] END learning_rate=0.5, max_depth=1, n_estimators=500;, score=0.267 total time=   1.2s
[CV 3/3; 17/120] START learning_rate=0.5, max_depth=3, n_estimators=300.........
[CV 3/3; 17/120] END learning_rate=0.5, max_depth=3, n_estimators=300;, score=0.180 total time=   0.9s
[CV 1/3; 18/120] START learning_rate=0.5, max_depth=3, n_estimators=500.........
[CV 1/3; 18/120] END learning_rate=0.5, max_depth=3, n_estimators=500;, score=0.054 total time=   1.4s
[CV 3/3; 33/120] START learning_rate=0.1, max_depth=1, n_estimators=100.........
[CV 3/3; 33/120] END learning_rate=0.1, max_depth=1, n_estimators=100;, score=0.214 total time=   0.3s
[CV 1/3; 34/120] START learning_rate=0.1, max_depth=1, n_estimators=200.........
[


[CV 3/3; 105/120] END learning_rate=0.015, max_depth=3, n_estimators=100;, score=-2.767 total time=   0.3s
[CV 2/3; 108/120] START learning_rate=0.015, max_depth=3, n_estimators=500......
[CV 2/3; 108/120] END learning_rate=0.015, max_depth=3, n_estimators=500;, score=-0.039 total time=   1.4s
[CV 3/3; 118/120] START learning_rate=0.015, max_depth=5, n_estimators=200......
[CV 3/3; 118/120] END learning_rate=0.015, max_depth=5, n_estimators=200;, score=-0.086 total time=   0.6s
[CV 3/3; 1/120] START learning_rate=0.5, max_depth=1, n_estimators=10...........
[CV 3/3; 1/120] END learning_rate=0.5, max_depth=1, n_estimators=10;, score=0.188 total time=   0.0s
[CV 2/3; 5/120] START learning_rate=0.5, max_depth=1, n_estimators=300..........
[CV 2/3; 5/120] END learning_rate=0.5, max_depth=1, n_estimators=300;, score=0.250 total time=   0.7s
[CV 1/3; 13/120] START learning_rate=0.5, max_depth=3, n_estimators=10..........
[CV 1/3; 13/120] END learning_rate=0.5, max_depth=3, n_estimators=10;,